In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import numpy as np # linear algebra
import pandas # data processing, CSV file I/O (e.g. pd.read_csv)
from pprint import pprint
import time
import io
import os
import re
from pprint import pprint
from IPython.core.display import HTML
from IPython.display import FileLink

g = 0

thisyear = 1917
league = 'A'

just_this_team = 'SEA' #['ANA', 'BAL']
just_this_team_order = 10

just_this_team_flag = False
# if (thisyear >= 1980):
#     just_this_team_flag = True

pandas.set_option('display.max_colwidth', None)

eventdir = '../input/baseball-events-from-retrosheetorg/events/'
processed_data_dir = '../input/baseball-events-from-retrosheetorg/'
rosters_dir = '../input/baseball-events-from-retrosheetorg/'
#rosters_dir = '../working/'

save_fielders = True

print('initializing dataframes...')
event_id_columns = ['eventID']
gamecolumns = ['gameID']
info_columns = gamecolumns + ['visteam', 'hometeam', 'site', 'date', 'number', 'starttime', 'daynight', 'usedh', 'innings', 'tiebreaker', 'umphome', 'ump1b', 'ump2b', 'ump3b', 'umplf', 'umprf', 'inputtime', 'howscored', 'pitches', 'oscorer', 'temp', 'winddir', 'windspeed', 'fieldcond', 'precip', 'sky', 'timeofgame', 'attendance', 'wp', 'lp', 'save', 'comments']
player_columns = ['playerID', 'visitor_or_home', 'batting_order', 'fielding_position']

event_columns = event_id_columns + gamecolumns
event_columns += ['event_in_game', 'inning', 'visitor_or_home', 'batterID', 'batter_hand']
event_columns += ['pitch_count', 'pitch_sequence', 'theplay']
event_columns += ['pitcherID', 'pitcher_hand', 'badj', 'padj', 'ladj', 'presadj']
if (save_fielders):
    event_columns += ['field_C_playerID', 'field_1B_playerID', 'field_2B_playerID', 'field_3B_playerID', 'field_SS_playerID', 'field_LF_playerID', 'field_CF_playerID', 'field_RF_playerID', 'bat_DH_playerID']
event_columns += ['pinch_runner_playerID', 'pinch_runner_replaced_playerID']#, 'badj', 'padj']
event_columns += ['comments']

def ordinaltg(n):
    return str(n) + {1: 'st', 2: 'nd', 3: 'rd'}.get(4 if 10 <= n % 100 < 20 else n % 10, "th")

os.chdir(r'/kaggle/working')

g = 0
print('Packages loaded!')

initializing dataframes...
Packages loaded!


In [5]:
s = time.time()

#print('restarting kernel...')
#HTML("<script>Jupyter.notebook.kernel.restart()</script>")

show_all_games = False
show_all_lines = False

start_at_line = 0

game_info_all_df = pandas.DataFrame(data=None, columns=info_columns).set_index('gameID')
game_data_all_df = pandas.DataFrame(data=None, columns=['gameID'])#.set_index('gameID')
er_all_df = pandas.DataFrame(data=None, columns=['gameID', 'pitcher', 'er'])

events_all_df = pandas.DataFrame(data=None, columns=event_columns).set_index('eventID')

players_all_df = pandas.DataFrame(data=None, columns=player_columns).set_index(['playerID'])


for q in range(thisyear, thisyear+1):
    game_info_thisyear_df = pandas.DataFrame(data=None, columns=game_info_all_df.columns)
#    game_data_thisyear_df = pandas.DataFrame(data=None, columns=game_data_all_df.columns)
    er_thisyear_df = pandas.DataFrame(data=None, columns=er_all_df.columns)
    players_thisyear_df = pandas.DataFrame(data=None, columns=players_all_df.columns)
    events_thisyear_df = pandas.DataFrame(data=None, columns=events_all_df.columns)
    
    thisyear_files = sorted(['{0:}{1:.0f}seve/'.format(eventdir, np.floor(q/10)*10)+x for x in os.listdir('{0:}{1:.0f}seve/'.format(eventdir, np.floor(q/10)*10)) if ((str(q) in x) & (x[-4:] == '.EV{0:}'.format(league)))])
    print('Found {0:,.0f} event files...'.format(len(thisyear_files)))
    
    if (just_this_team_flag):
        start_at_file_number = just_this_team_order
        end_at_file_number = just_this_team_order + 1
    else:
        start_at_file_number = 0
        #end_at_file_number = 1
        end_at_file_number = len(thisyear_files)
        
    for thisfile in thisyear_files[start_at_file_number:end_at_file_number]:
        game_info_df = pandas.DataFrame(data=None, columns=game_info_thisyear_df.columns)
#        game_data_df = pandas.DataFrame(data=None, columns=game_data_thisyear_df.columns)
        er_df = pandas.DataFrame(data=None, columns=er_thisyear_df.columns)
        events_df = pandas.DataFrame(data=None, columns=events_thisyear_df.columns)
        players_df = pandas.DataFrame(data=None, columns=players_thisyear_df.columns)

        comments_at_events = []
        pinch_runners_at_events = []
        badj_at_events = []
        padj_at_events = []
        ladj_at_events = []
        presadj_at_events = []

        all_lines_list = []
#         if (thisfile.split('/')[-1][4:7] == 'SLA'):
#             show_all_lines = True
#         else:
#             show_all_lines = False
        print('Reading {0:}...'.format(thisfile.split('/')[-1]))
        with io.open(thisfile, 'r', encoding='utf-8') as f:
            orig_lines_list = f.readlines()
        for this_orig_line in orig_lines_list:
            all_lines_list += [x for x in this_orig_line.split('\n')]
        all_lines_list = [x for x in all_lines_list 
                          if ((x != '') and (x != '\x1a'))
                         ]
        #print('\tRead {0:,.0f} lines...'.format(len(all_lines_list)))
        
        event_dict = {}
        gamecount = 0
        eventcount = 0
        the_event_in_game = 0
        batting_team = 0
        comment_text = ''
        game_info_comment_text = ''

        [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [False, np.nan, np.nan]
        staged_play = False
        
        for i in range(start_at_line, len(all_lines_list)): #range(0, len(all_lines_list)):
#             if (i > 11910):
#                 break
            if (show_all_lines):
                print('\tParsing line {0:,.0f} of {1:.0f}...'.format(i, len(all_lines_list)))
                print(i, all_lines_list[i])
            elif (np.mod(i,1000) == 0):
                print('\tParsing line {0:,.0f} of {1:,.0f}...'.format(i, len(all_lines_list)))
            record_type = all_lines_list[i].split(',')[0]
            #print(record_type)
            if (record_type == 'id'):   
                watchflag = False
#                 if (all_lines_list[i].split(',')[1] == 'PHA191807062'):
#                     watchflag = True
                if (gamecount >= 1):   #WRITE LAST PLAY OF PREVIOUS GAME!
                    if (comment_text != ''):
                        event_dict['comments'] = comment_text
                    else:
                        event_dict['comments'] = np.nan
                    event_dict['pinch_runner_playerID'] = pinch_runner
                    event_dict['pinch_runner_replaced_playerID'] = pinch_runner_replaced
                    if (pinch_runner_flag == True):
                        [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [False, np.nan, np.nan]
                    events_df.loc[eventcount] = event_dict
                    eventcount += 1
                    the_event_in_game == 1
                    comment_text = ''

                    if (game_info_comment_text != ''):
                        this_game_info['comments'] = game_info_comment_text
                        game_info_comment_text = ''
                    else:
                        this_game_info['comments'] = np.nan
                    game_info_df.loc[thegameID] = this_game_info

                gamecount += 1        
                thegameID = all_lines_list[i].split(',')[1]
                if (show_all_games):
                    print('\t\tReading {0:} (starts on line {1:.0f})...'.format(thegameID, i))
                elif (np.mod(gamecount, 50) == 0):
                    print('\t\tReading {0:} (starts on line {1:.0f})...'.format(thegameID, i))

                players_df = players_df[players_df['visitor_or_home'].apply(lambda x: True == False)]

        #         if (thegameID == 'ANA202008100'):
        #             break
                eventcount += 1
                this_game_info = {}
                the_event_in_game = 1
                staged_play = False
#                 if (np.mod(gamecount, 50) == 1):
#                     print('\t\tParsing game {0:.0f}: {1:} (i = {2:,.0f}; eventcount = {3:,.0f})...'.format(gamecount, thegameID, i, eventcount))
        #            print('Starting Top of the 1st...')
            elif (record_type == 'version'):
                this_game_info['version'] = all_lines_list[i].split(',')[1]
            elif (record_type == 'info'):
                this_game_info[all_lines_list[i].split(',')[1]] = all_lines_list[i].split(',')[2]
            elif (record_type == 'start'):
                this_player_df = pandas.DataFrame(data=[[all_lines_list[i].split(',')[1], int(all_lines_list[i].split(',')[3]), int(all_lines_list[i].split(',')[4]), int(all_lines_list[i].split(',')[5])]], columns=player_columns).set_index('playerID')
                players_df = pandas.concat((players_df, this_player_df), axis=0)
            elif (record_type == 'data'):
                try:
                    this_er_row_df = pandas.DataFrame(data=[[thegameID, all_lines_list[i].split(',')[2], int(all_lines_list[i].split(',')[3])]], columns=er_df.columns)
                except ValueError:
                    this_er_row_df = pandas.DataFrame(data=[[thegameID, all_lines_list[i].split(',')[2], int(all_lines_list[i].split(',')[3][0:2])]], columns=er_df.columns) # Catching one line (BAL2021 line 6015) with a stray "1    a"
                er_df = pandas.concat((er_df, this_er_row_df))
            elif (record_type == 'play'):
                if ((watchflag) & (all_lines_list[i].split(',')[-1] == 'NP')):
                    print('{0:0d}: {1:}: {2:}'.format(i, all_lines_list[i], all_lines_list[i].split(',')[-1]))
                if (staged_play):
                    if (comment_text != ''):
                        event_dict['comments'] = comment_text
                    else:
                        event_dict['comments'] = np.nan
                    event_dict['pinch_runner_playerID'] = pinch_runner
                    event_dict['pinch_runner_replaced_playerID'] = pinch_runner_replaced
                    if (pinch_runner_flag == True):
                        [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [False, np.nan, np.nan]
                    events_df.loc[eventcount] = event_dict
                    eventcount += 1
                    the_event_in_game == 1
                    comment_text = ''
                event_dict['event_in_game'] = the_event_in_game
                event_dict['gameID'] = thegameID
                event_dict['inning'] = int(all_lines_list[i].split(',')[1])
                event_dict['visitor_or_home'] = int(all_lines_list[i].split(',')[2])
                event_dict['batterID'] = all_lines_list[i].split(',')[3]
                event_dict['pitch_count'] = all_lines_list[i].split(',')[4]
                event_dict['pitch_sequence'] = all_lines_list[i].split(',')[5]
                event_dict['theplay'] = all_lines_list[i].split(',')[6]        
                
                #print(players_df)
                
                event_dict['pitcherID'] = players_df[
                    (players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) 
                    & (players_df['fielding_position'] == 1)
                ].index.values[0]
                #print('\t{0:}'.format(event_dict['pitcherID']))
                if (save_fielders):
                    event_dict['field_C_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 2)].index.values[0]
                    event_dict['field_1B_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 3)].index.values[0]
                    event_dict['field_2B_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 4)].index.values[0]
                    event_dict['field_3B_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 5)].index.values[0]
                    event_dict['field_SS_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 6)].index.values[0]
                    event_dict['field_LF_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 7)].index.values[0]
                    event_dict['field_CF_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 8)].index.values[0]
                    event_dict['field_RF_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 9)].index.values[0]
                                        
#                     if (this_game_info['usedh'] == 'true'):  # lol it's a string, not a boolean
#                         event_dict['bat_DH_playerID'] = players_df[(players_df['visitor_or_home'] == event_dict['visitor_or_home']) & (players_df['fielding_position'] == 10)].index.values[0]
                if (batting_team != event_dict['visitor_or_home']):
        #             if (event_dict['visitor_or_home'] == 0):
        #                 inningside = 'Top'
        #             else:
        #                 inningside = 'Bottom'
        #             print('Starting {0:} of the {1:} - {2:} pitching...'.format(inningside, ordinal(event_dict['inning']), event_dict['pitcherID']))
                    batting_team = 1 - batting_team
                staged_play = True
                the_event_in_game += 1
            elif (record_type == 'com'):
                if (the_event_in_game <= 1):
                    game_info_comment_text += all_lines_list[i][4:]
                if (eventcount not in comments_at_events):
                    comments_at_events.append(eventcount)
                if (staged_play):
                    if (comment_text != ''):
                        comment_text += ' '
                    comment_text += all_lines_list[i][4:]#.replace('"', '').replace('$', '').replace('.', '')
            elif (record_type == 'sub'):
                players_df.index.name = 'playerID'
                players_df = players_df.reset_index()
                sub_playerID = all_lines_list[i].split(',')[1]
                if (len(all_lines_list[i].split(',')) == 6):
                    sub_team = int(all_lines_list[i].split(',')[3])
                    sub_batting_order = int(all_lines_list[i].split(',')[4])
                    sub_fielding_position = int(all_lines_list[i].split(',')[5])
                else: # Replace commas in substituted-in player's name
                    commafinder = ''
                    for q in range(2, len(all_lines_list[i].split(','))):
                        try:
                            sub_team = int(all_lines_list[i].split(',')[q])
                            sub_batting_order = int(all_lines_list[i].split(',')[q+1])
                            sub_fielding_position = int(all_lines_list[i].split(',')[q+2])
                            commafinder = commafinder.replace('"','')[:-1]
                            print('\t\tReplacing comma in name of {0:} ({1:})...'.format(commafinder, sub_playerID))
#                             print(commafinder)
#                             print(sub_team)
#                             print(sub_batting_order)
#                             print(sub_fielding_position)
                            break
                        except ValueError:
                                commafinder += all_lines_list[i].split(',')[q] + ' '
#                 if (sub_playerID == 'rices101'):
#                     print('*********************')
#                     print('')
#                     print('sub_playerID',sub_playerID)
#                     print('sub_team = ',sub_team, ' and batting_team = ',batting_team)
#                     print('sub_batting_order = ',sub_batting_order)
#                     print('sub_fielding_position = ',sub_fielding_position)
                                
                if (sub_team == batting_team):
                    if (sub_fielding_position == 11):
#                        print('\tPINCH HITTER at line {0:.0f}: {1:} in at batting order {2:.0f}, replacing {3:}!!!!!!'.format(i, sub_playerID, sub_batting_order, players_df[(players_df['visitor_or_home'] == sub_team) & (players_df['batting_order'] == sub_batting_order)]['playerID'].values[0]))
                        players_df.loc[(players_df['visitor_or_home'] == batting_team) & (players_df['batting_order'] == sub_batting_order), 'playerID'] = sub_playerID
#                        print('is now {0:}'.format(players_df[(players_df['visitor_or_home'] == sub_team) & (players_df['batting_order'] == sub_batting_order)]['playerID'].values[0]))
                        #event_dict['batterID'] = sub_playerID
                    elif (sub_fielding_position == 12):
                        #print('\tPINCH RUNNER at line {0:.0f}!!!!!!'.format(i))
                        pinch_runners_at_events.append(eventcount)
                        [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [True, sub_playerID, players_df[(players_df['visitor_or_home'] == batting_team) & (players_df['batting_order'] == sub_batting_order)]['playerID'].values[0]]
                        #print([pinch_runner_flag, pinch_runner, pinch_runner_replaced])
                    else:
                        players_df.loc[(players_df['visitor_or_home'] == batting_team) & (players_df['batting_order'] == sub_batting_order), 'playerID'] = sub_playerID
                else:
#                     if (sub_playerID == 'rices101'):
#                         print(players_df[(players_df['visitor_or_home'] == 1-batting_team) & (players_df['fielding_position'] == sub_fielding_position)])
                    players_df.loc[
                        (players_df['visitor_or_home'] == 1-batting_team) 
                        & (players_df['fielding_position'] == sub_fielding_position), 'playerID'] = sub_playerID
#                     if (sub_playerID == 'rices101'):
#                         print(players_df[(players_df['visitor_or_home'] == 1-batting_team) & (players_df['fielding_position'] == sub_fielding_position)])
#                         print('*********************')
#                         print('')
                players_df = players_df.set_index('playerID')
            elif (record_type == 'radj'):
                pinch_runners_at_events.append(eventcount)
        #         print('-------------------- RADJ ---------------------------')
        #         print(all_lines_list[i])
                [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [True, all_lines_list[i].split(',')[1], 'radj']
        #         print(pinch_runner_flag, pinch_runner, pinch_runner_replaced)
        #         print('-------------------- /RADJ ---------------------------')
            elif (record_type == 'badj'):

                if (the_event_in_game == 1):
                    #print('\tbadj at eventcount {0:,.0f}: {1:}'.format(eventcount, all_lines_list[i]))
                    badj_at_events.append(eventcount)
                else:
                    #print('\tbadj at eventcount {0:,.0f}: {1:}'.format(eventcount+1, all_lines_list[i]))
                    badj_at_events.append(eventcount+1)
            elif (record_type == 'padj'):
                if (the_event_in_game == 1):
                    #print('\tpadj at eventcount {0:,.0f}: {1:}'.format(eventcount, all_lines_list[i]))
                    padj_at_events.append(eventcount)
                else:
                    #print('\tpadj at eventcount {0:,.0f}: {1:}'.format(eventcount, all_lines_list[i]))
                    padj_at_events.append(eventcount+1)
            elif (record_type == 'ladj'):
                #print('\tladj at eventcount {0:,.0f}: {1:}'.format(eventcount, all_lines_list[i]))
                #print(all_lines_list[i])
                ladj_at_events.append(eventcount)
            elif (record_type == 'presadj'):
                presadj_at_events.append(eventcount)
            else:
                print('--------------')
                print(all_lines_list[i-1])     
                print(all_lines_list[i])     
                print(all_lines_list[i+1])     
                print('--------------')
                print('--------------')

        # record last play of final game
        if (comment_text != ''):
            event_dict['comments'] = comment_text
        else:
            event_dict['comments'] = np.nan
        event_dict['pinch_runner_playerID'] = pinch_runner
        event_dict['pinch_runner_replaced_playerID'] = pinch_runner_replaced
        if (pinch_runner_flag == True):
            [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [False, np.nan, np.nan]
        events_df.loc[eventcount] = event_dict
        eventcount += 1
        the_event_in_game == 1
        comment_text = ''        

        # record final game info and data
        if (game_info_comment_text != ''):
            this_game_info['comments'] = game_info_comment_text
        else:
            this_game_info['comments'] = np.nan
        game_info_df.loc[thegameID] = this_game_info
        events_df.loc[badj_at_events, 'badj'] = True        
        events_df.loc[padj_at_events, 'padj'] = True
        events_df.loc[ladj_at_events, 'ladj'] = True
        events_df.loc[presadj_at_events, 'presadj'] = True
        
        game_info_thisyear_df = pandas.concat((game_info_thisyear_df, game_info_df), axis=0)
#        game_data_thisyear_df = pandas.concat((game_data_thisyear_df, game_data_df), axis=0)
        er_thisyear_df = pandas.concat((er_thisyear_df, er_df), axis=0)
        players_thisyear_df = pandas.concat((players_thisyear_df, players_df), axis=0)
        
        events_thisyear_df = pandas.concat((events_thisyear_df, events_df), axis=0)

        print('\n')
        
    game_info_all_df = pandas.concat((game_info_all_df, game_info_thisyear_df), axis=0)
#    game_data_all_df = pandas.concat((game_data_all_df, game_data_thisyear_df), axis=0)
    er_all_df = pandas.concat((er_all_df, er_thisyear_df), axis=0)
    players_all_df = pandas.concat((players_all_df, players_thisyear_df), axis=0)

    events_all_df = pandas.concat((events_all_df, events_thisyear_df), axis=0)

    
#print('Killing in-progress events and replacing with the full file...')

game_info_df = game_info_df[game_info_df['number'].apply(lambda x: True == False)]
er_df = er_df[er_df['er'].apply(lambda x: True == False)]
players_df = players_df[players_df['visitor_or_home'].apply(lambda x: True == False)]
events_df = events_df[events_df['event_in_game'].apply(lambda x: True == False)]

game_info_df = game_info_all_df
er_df = er_all_df
players_df = players_all_df
events_df = events_all_df

er_df = er_df.reset_index(drop=True)
events_df = events_df.reset_index(drop=True)

for x in ['badj', 'padj', 'ladj', 'presadj']:
    events_df.loc[:, x] = events_df[x].fillna(False)

game_info_df.index.name = 'gameID'
er_df.index.name = 'erID'
players_df.index.name = 'playerID'
events_df.index.name = 'eventID'

# print('backing up...')
game_info_df_bk = game_info_df
er_df_bk = er_df
players_df_bk = players_df
events_df_bk = events_df

e = time.time()
g = g + (e-s) 
#print('\n')
print('Parsed {0:,.0f} events in {1:,.0f} minutes {2:,.0f} seconds!'.format(len(events_df), np.floor((e-s)/60), (e-s)%60))


Found 8 event files...
Reading 1917BOS.EVA...
	Parsing line 0 of 6,647...
	Parsing line 1,000 of 6,647...
	Parsing line 2,000 of 6,647...
	Parsing line 3,000 of 6,647...
	Parsing line 4,000 of 6,647...
	Parsing line 5,000 of 6,647...
	Parsing line 6,000 of 6,647...


Reading 1917CHA.EVA...
	Parsing line 0 of 5,957...
	Parsing line 1,000 of 5,957...
	Parsing line 2,000 of 5,957...
	Parsing line 3,000 of 5,957...
	Parsing line 4,000 of 5,957...
	Parsing line 5,000 of 5,957...


Reading 1917CLE.EVA...
	Parsing line 0 of 9,619...
	Parsing line 1,000 of 9,619...
	Parsing line 2,000 of 9,619...
	Parsing line 3,000 of 9,619...
	Parsing line 4,000 of 9,619...
	Parsing line 5,000 of 9,619...
	Parsing line 6,000 of 9,619...
	Parsing line 7,000 of 9,619...
		Reading CLE191708182 (starts on line 7117)...
	Parsing line 8,000 of 9,619...
	Parsing line 9,000 of 9,619...


Reading 1917DET.EVA...
	Parsing line 0 of 5,527...
	Parsing line 1,000 of 5,527...
	Parsing line 2,000 of 5,527...
	Parsing line 3

In [6]:
def add_roster_entry_if_needed(rosters_df, theplayerID, thenameLast, thenameFirst, thebatting_hand, thethrowing_hand, theteam, theposition, thetheyear):
    if not(theplayerID in rosters_df['playerID'].tolist()):
        rosters_df.loc[rosters_df.index.max()+1] = [theplayerID, thenameLast, thenameFirst, thebatting_hand, thethrowing_hand, theteam, theposition, thetheyear]
        #print('{0:} ({1:} {2:}) already in rosters file...'.format(theplayerID, thenameFirst, thenameLast))
        #pprint(rosters_df[rosters_df['playerID'] == theplayerID])
        #print('----------------------------')
        
    return rosters_df

def add_special_case_roster_entries(rosters_df):
### 1914
## FL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'tappw101', 'Tappan', 'Walter', 'R', 'R', 'KCF', 'SS', 1914)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yound104', 'Young', 'Delmer', 'L', 'R', 'BUF', 'RF', 1914)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zwild101', 'Zwilling', 'Edward', 'L', 'L', 'CHF', 'CF', 1914)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yound104', 'Young', 'Delmer', 'L', 'R', 'BUF', 'RF', 1914)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zinng101', 'Zinn', 'Guy', 'L', 'R', 'BLF', 'RF', 1914)
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'woodr103', 'Wood', 'Roy', 'R', 'R', 'CLE', 'RF', 1914)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wyckj101', 'Wyckoff', 'John', 'R', 'R', 'PHA', 'P', 1914)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zeism101', 'Zieser', 'Mathias', 'R', 'R', 'BOS', 'P', 1914)
### 1915
## FL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'dougl101', 'Douglas', 'Lawrence', 'R', 'R', 'BLF', 'P', 1915)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wrigd101', 'Wright', 'Willard', 'R', 'R', 'BRF', 'C', 1915)
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'connt101', 'Connolly', 'Thomas', 'L', 'R', 'WS1', 'RF', 1915)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'conwo101', 'Conway', 'Owen', '?', 'R', 'PHA', '3B', 1915)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'fluhj101', 'Fluhrer', 'John', 'R', 'R', 'CHN', 'LF', 1915)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zimmb101', 'Zimmerman', 'William', 'R', 'R', 'BRO', 'RF', 1915)
### 1916
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'carpp101', 'Carpenter', 'Paul', 'R', 'R', 'PIT', 'P', 1916)
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wrigc101', 'Wright', 'Ceylon', 'R', 'R', 'CHA', 'SS', 1916)
### 1917
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'waldd101', 'Waldbauer', 'Albert', 'R', 'R', 'WS1', 'P', 1917)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wrigr104', 'Wright', 'Wayne', 'R', 'R', 'SLA', 'P', 1917)
### 1918
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'fahef101', 'Fahey', 'Francis', 'B', 'R', 'PHA', 'LF', 1918)    
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wheed102', 'Wheeler', 'Richard', 'R', 'R', 'SLN', 'C', 1918)
### 1919
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zinnj101', 'Zinn', 'James', 'L', 'R', 'PHA', 'P', 1919)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'fishs102', 'Fishburn', 'Samuel', 'R', 'R', 'SLN', 'C', 1919)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'gerne101', 'Gerner', 'Edwin', 'L', 'L', 'CIN', 'P', 1919)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yeabb101', 'Yeabsley', 'Robert', 'R', 'R', 'PHI', 'C', 1919)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zitzb101', 'Zitzmann', 'William', 'R', 'R', 'CIN', 'C', 1919)
### 1920
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'conwj101', 'Conway', 'Jerome', 'L', 'L', 'WS1', 'P', 1920)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'ellig101', 'Ellison', 'George', 'R', 'R', 'CLE', 'C', 1920)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'davec101', 'Davenport', 'Claude', 'R', 'R', 'NY1', 'P', 1920)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'griet102', 'Griesenbeck', 'Carlos', 'R', 'R', 'SLN', 'C', 1920)
### 1921
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'freeh102', 'Freeman', 'Harvey', 'R', 'R', 'PHA', 'P', 1921)
### 1922
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'cox-e101', 'Cox', 'Ernest', 'L', 'R', 'CHA', 'P', 1922)
### 1922
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'eubau101', 'Eubanks', 'Uel', 'R', 'R', 'CHN', 'P', 1922)
### 1923
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'woodd104', 'Wood', 'Charles', 'R', 'R', 'PHA', 'SS', 1923)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wigif101', 'Wigington', 'Fred', 'R', 'R', 'SLN', 'P', 1923)
### 1924
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'gibsc101', 'Gibson', 'Charles', 'R', 'R', 'PHA', 'C', 1924)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'workh102', 'Workman', 'Harry', 'R', 'R', 'BOS', 'P', 1924)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yowec101', 'Yowell', 'Carl', 'L', 'L', 'CLE', 'P', 1924)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yde-e101', 'Yde', 'Emil', 'B', 'L', 'PIT', 'P', 1924)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yeara101', 'Yeargin', 'James', 'R', 'R', 'BSN', 'P', 1924)
### 1926
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'uchrj101', 'Uchrinscko', 'James', 'L', 'R', 'WS1', 'P', 1926)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yarnr101', 'Yarnall', 'Waldo', 'R', 'R', 'PHI', 'C', 1926)
### 1927
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'browj102', 'Brown', 'Joseph', 'R', 'R', 'CHA', 'P', 1927)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yerkc101', 'Yerkes', 'Charles', 'R', 'L', 'PHA', 'P', 1927)
### 1928
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'grifm102', 'Griffin', 'Martin', 'R', 'R', 'BOS', 'P', 1928)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wilsr101', 'Wilson', 'Roy', 'L', 'L', 'CHA', 'C', 1928)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'greej103', 'Greene', 'Julius', 'L', 'R', 'PHI', 'P', 1928)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wille102', 'Williams', 'Earl', 'R', 'R', 'BSN', 'C', 1928)
### 1929
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'carre101', 'Carroll', 'Edgar', 'R', 'R', 'BOS', 'P', 1929)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'cronj103', 'Cronin', 'James', 'B', 'R', 'PHA', '2B', 1929)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'westa101', 'Weston', 'Alfred', 'R', 'R', 'BSN', 'P', 1929)
### 1930
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'chilh101', 'Child', 'Harry', 'B', 'R', 'WS1', 'P', 1930)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'woods101', 'Wood', 'Charles', 'L', 'L', 'PIT', 'P', 1930)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wysob101', 'Wysong', 'Harlan', 'L', 'L', 'CIN', 'P', 1930)
### 1931
    rosters_df = add_roster_entry_if_needed(rosters_df, 'younr102', 'Young', 'Russell', 'B', 'R', 'SLA', 'C', 1931)
### 1932
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'edele101', 'Edelen', 'Edward', 'R', 'R', 'WS1', 'P', 1932)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wisea101', 'Wise', 'Archibald', 'R', 'R', 'CHA', 'P', 1932)
### 1933
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'chape101', 'Chapman', 'Edwin', 'B', 'R', 'WS1', 'P', 1933)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'coomb101', 'Coombs', 'Raymond', 'R', 'R', 'PHA', 'P', 1933)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zapuj101', 'Zapustas', 'Joseph', 'R', 'R', 'PHA', 'LF', 1933)
### 1934
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'carsk101', 'Carson', 'Walter', 'L', 'L', 'CLE', 'RF', 1934)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wilsw103', 'Wilshere', 'Vernon', 'L', 'L', 'PHA', 'P', 1934)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'chapg101', 'Chapman', 'Glenn', 'R', 'R', 'BRO', '2B', 1934)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'darrg101', 'Darrow', 'George', 'L', 'L', 'PHI', 'P', 1934)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wiedc101', 'Wiedemeyer', 'Charles', 'L', 'L', 'CHN', 'C', 1934)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wistw101', 'Wistert', 'Francis', 'R', 'R', 'CIN', 'C', 1934)
### 1935
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'hayej103', 'Hayes', 'James', 'L', 'R', 'WS1', 'P', 1935)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wriga101', 'Wright', 'Albert', 'R', 'R', 'CLE', 'RF', 1935)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wriga101', 'Wright', 'Albert', 'R', 'R', 'CLE', 'RF', 1935)
### 1936
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zubeb101', 'Zuber', 'William', 'R', 'R', 'CLE', 'P', 1936)
### 1937
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yound105', 'Young', 'Delmer', 'B', 'R', 'PHI', '2B', 1937)
### 1941
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'hajdc101', 'Hajduk', 'Chester', 'R', 'R', 'CHA', 'C', 1941)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'washg103', 'Washburn', 'George', 'L', 'R', 'NYA', 'C', 1941)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zienb101', 'Zientara', 'Benedict', 'R', 'R', 'CIN', '2B', 1941)
### 1942
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'heimv101', 'Heim', 'Val', 'L', 'R', 'CHA', 'LF', 1942)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'flith101', 'Flitcraft', 'Hildreth', 'L', 'L', 'PHI', 'P', 1942)
### 1943
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'woodp102', 'Woods', 'George', 'R', 'R', 'BOS', 'P', 1943)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'dagej101', 'Dagenhard', 'John', 'R', 'R', 'BSN', 'C', 1943)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'webbb102', 'Webb', 'Willie', 'R', 'R', 'PHI', 'P', 1943)
### 1944
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wilkb101', 'Wilkins', 'Robert', 'R', 'R', 'PHA', 'SS', 1944)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zinsb101', 'Zinser', 'William', 'R', 'R', 'WS1', 'P', 1944)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zolds101', 'Zoldak', 'Samuel', 'L', 'L', 'SLA', 'P', 1944)
##  NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'delat102', 'de la Cruz', 'Tomas', 'R', 'R', 'CIN', 'P', 1944)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'eastr102', 'Easterwood', 'Roy', 'R', 'R', 'CHN', 'C', 1944)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yorkt102', 'York', 'Tony', 'R', 'R', 'CHN', 'SS', 1944)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zachc102', 'Zachary', 'Albert', 'R', 'R', 'BRO', 'P', 1944)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zak-f101', 'Zak', 'Frank', 'R', 'R', 'PIT', 'SS', 1944)
### 1945
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'grayp101', 'Gray', 'Peter', 'L', 'L', 'SLA', 'LF', 1945)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'willp102', 'Williams', 'Fred', 'R', 'R', 'CLE', '1B', 1945)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zardj101', 'Zardon', 'Jose', 'R', 'R', 'WS1', 'CF', 1945)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'whitb106', 'White', 'William', 'R', 'R', 'BRO', '3B', 1945)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zimmr101', 'Zimmerman', 'Roy', 'L', 'L', 'NY1', '1B', 1945)
### 1946
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'deutm101', 'Deutsch', 'Melvin', 'R', 'R', 'BOS', 'P', 1946)
### 1948
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wilsg103', 'Wilson', 'Grady', 'R', 'R', 'PIT', 'SS', 1948)
### 1950
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'workh101', 'Workman', 'Henry', 'L', 'R', 'NYA', 'C', 1950)
### 1951
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zaucn101', 'Zauchin', 'Norbert', 'R', 'R', 'BOS', '1B', 1951)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zuveg101', 'Zuverink', 'George', 'R', 'R', 'CLE', 'P', 1951)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'dempc101', 'Dempsey', 'Cornelius', 'R', 'R', 'PIT', 'P', 1951)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yochl101', 'Yochim', 'Leonard', 'L', 'L', 'PIT', 'P', 1951)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yound101', 'Young', 'Richard', 'L', 'R', 'PHI', '2B', 1951)
### 1952
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'clarm101', 'Clark', 'Michael', 'R', 'R', 'SLN', 'P', 1952)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'davib103', 'Davis', 'Robert', 'R', 'R', 'PIT', 'RF', 1952)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wolfe101', 'Wolfe', 'Edward', 'R', 'R', 'PIT', 'P', 1952)
### 1953
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'carsf101', 'Carswell', 'Frank', 'R', 'R', 'DET', 'LF', 1953)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'davay101', 'Davalillo', 'Pompeyo', 'R', 'R', 'WS1', 'SS', 1953)
### 1954
## AL    
    rosters_df = add_roster_entry_if_needed(rosters_df, 'bullg101', 'Bullard', 'George', 'R', 'R', 'DET', 'C', 1954)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zickb101', 'Zick', 'Robert', 'L', 'R', 'CHN', 'P', 1954)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zimmd101', 'Zimmer', 'Donald', 'R', 'R', 'BRO', 'SS', 1954)
### 1955
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'delij101', 'Delis', 'Juan', 'R', 'R', 'WS1', '3B', 1955)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'white106', 'White', 'Edward', 'R', 'R', 'CHA', 'RF', 1955)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'watef101', 'Waters', 'Fred', 'L', 'L', 'PIT', 'C', 1955)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'westj101', 'Westlake', 'James', 'L', 'L', 'PHI', 'C', 1955)
### 1956
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wrigr101', 'Wright', 'Roy', 'R', 'R', 'NY1', 'C', 1956)
### 1957
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yewct101', 'Yewcic', 'Thomas', 'R', 'R', 'DET', 'C', 1957)
### 1958
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wilsd101', 'Wilson', 'Duane', 'L', 'L', 'BOS', 'P', 1958)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zannd101', 'Zanni', 'Dominick', 'R', 'R', 'SFN', 'P', 1958)
### 1960
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wilsn101', 'Wilson', 'Samuel', 'L', 'R', 'SFN', 'C', 1960)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'woodj103', 'Woods', 'James', 'R', 'R', 'PHI', '3B', 1960)
### 1961
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'woodj107', 'Wood', 'Jacob', 'R', 'R', 'DET', '2B', 1961)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yastc101', 'Yastrzemski', 'Carl', 'L', 'R', 'BOS', 'LF', 1961)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zipfb101', 'Zipfel', 'Marion', 'L', 'R', 'WS2', '1B', 1961)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zimmj101', 'Zimmerman', 'Gerald', 'R', 'R', 'CIN', 'C', 1961)
### 1963
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'dustb101', 'Dustal', 'Robert', 'R', 'R', 'DET', 'P', 1963)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'cardc101', 'Cardinal', 'Conrad', 'R', 'R', 'HOU', 'P', 1963)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'stana101', 'Stanek', 'Albert', 'L', 'L', 'SFN', 'P', 1963)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'withc101', 'Withrow', 'Raymond', 'R', 'R', 'SLN', 'LF', 1963)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'woodw101', 'Woodward', 'William', 'R', 'R', 'MLN', 'SS', 1963)
### 1964
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'grayd103', 'Gray', 'David', 'R', 'R', 'BOS', 'P', 1964)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'fishf101', 'Fisher', 'Frederick', 'L', 'L', 'DET', 'P', 1964)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'vined101', 'Vineyard', 'David', 'R', 'R', 'BAL', 'P', 1964)
### 1965
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wantd101', 'Wantz', 'Richard', 'R', 'R', 'CAL', 'P', 1965)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yound103', 'Young', 'Donald', 'R', 'R', 'CHN', 'CF', 1965)
### 1967
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wynnb102', 'Wynne', 'Billy', 'L', 'R', 'NYN', 'P', 1967)
### 1968
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'coltl101', 'Colton', 'Lawrence', 'L', 'R', 'PHI', 'C', 1968)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'fastd101', 'Fast', 'Darcy', 'L', 'L', 'CHN', 'P', 1968)
### 1969
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'cridj101', 'Crider', 'Jerry', 'R', 'R', 'MIN', 'P', 1969)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'woodr102', 'Woods', 'Ronald', 'R', 'R', 'DET', 'LF', 1969)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'woodr102', 'Woods', 'Ronald', 'R', 'R', 'NYA', 'CF', 1969)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zeppb101', 'Zepp', 'William', 'R', 'R', 'MIN', 'P', 1969)
### 1970
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'colsl101', 'Colson', 'Loyd', 'R', 'R', 'NYA', 'P', 1970)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yorkj101', 'York', 'James', 'R', 'R', 'KCA', 'C', 1970)    
### 1971
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yatea101', 'Yates', 'Albert', 'R', 'R', 'MIL', 'RF', 1971)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'younl101', 'Yount', 'Lawrence', 'R', 'R', 'HOU', 'P', 1971)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'ziskr101', 'Zisk', 'Richard', 'R', 'R', 'PIT', 'LF', 1971)
### 1972
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'colec102', 'Coletta', 'Christopher', 'L', 'L', 'CAL', 'LF', 1972)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yanch101', 'Yancy', 'Hugh', 'R', 'R', 'CHA', '3B', 1972)    
### 1974
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zamoo101', 'Zamora', 'Oscar', 'R', 'R', 'CHN', 'P', 1974)
### 1977
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'cortb101', 'Cort', 'Barry', 'R', 'R', 'MIL', 'P', 1977)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'willm105', 'Williams', 'Mark', 'L', 'L', 'OAK', 'RF', 1977)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zdebj101', 'Zdeb', 'Joseph', 'R', 'R', 'KCA', 'LF', 1977)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zebeg101', 'Zeber', 'George', 'B', 'R', 'NYA', '2B', 1977)    
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zachp001', 'Zachry', 'Patrick', 'R', 'R', 'CIN', 'P', 1977)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zachp001', 'Zachry', 'Patrick', 'R', 'R', 'CIN', 'P', 1977)
### 1978
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'cripd101', 'Cripe', 'David', 'R', 'R', 'KCA', '3B', 1978)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'woodd103', 'Woodard', 'Darrell', 'R', 'R', 'OAK', '2B', 1978)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yuraj101', 'Yurak', 'Jeffrey', 'B', 'R', 'MIL', 'C', 1978)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'younk101', 'Young', 'Kip', 'R', 'R', 'DET', 'P', 1978)
### 1979
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'vasqr101', 'Vasquez', 'Rafael', 'R', 'R', 'SEA', 'P', 1979)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'davis102', 'Davis', 'Steven', 'R', 'R', 'CHN', '3B', 1979)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wilsg104', 'Wilson', 'Gary', 'R', 'R', 'HOU', 'P', 1979)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'white003', 'Whited', 'Edward', 'R', 'R', 'ATL', '3B', 1989)
### 1980
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'combg101', 'Combe', 'Geoffrey', 'R', 'R', 'CIN', 'P', 1980)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'figuj101', 'Figueroa', 'Jesus', 'L', 'L', 'CHN', 'CF', 1980)
### 1981
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'garcd101', 'Garcia', 'Daniel', 'L', 'L', 'KCA', 'RF', 1981)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'willd101', 'Williams', 'Dallas', 'L', 'L', 'BAL', 'LF', 1981)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wrigj101', 'Wright', 'James', 'R', 'R', 'KCA', 'P', 1981)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'woodg001', 'Woods', 'Gary', 'R', 'R', 'HOU', 'RF', 1981)
### 1982
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zuvep001', 'Zuvella', 'Paul', 'R', 'R', 'ATL', 'C', 1982)
### 1984
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zaskj001', 'Zaske', 'Lloyd', 'R', 'R', 'PIT', 'P', 1984)
### 1985
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'ferrt001', 'Ferreira', 'Anthony', 'L', 'L', 'KCA', 'P', 1985)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'woodb001', 'Woodward', 'Robert', 'R', 'R', 'BOS', 'P', 1985)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'youmf001', 'Youmans', 'Floyd', 'R', 'R', 'MON', 'P', 1985)
### 1987
    rosters_df = add_roster_entry_if_needed(rosters_df, 'dowek001', 'Dowell', 'Kenneth', 'R', 'R', 'PHI', 'SS', 1987)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'ziems001', 'Ziem', 'Stephen', 'R', 'R', 'ATL', 'P', 1987)
### 1988
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wortc001', 'Worthington', 'Craig', 'R', 'R', 'BAL', '3B', 1988)
### 1989
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'datzj001', 'Datz', 'Jeffrey', 'R', 'R', 'DET', 'C', 1989)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'willd001', 'Williams', 'Dana', 'R', 'R', 'BOS', 'LF', 1989)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wintm001', 'Winters', 'Matthew', 'L', 'R', 'KCA', 'RF', 1989)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zavac001', 'Zavaras', 'Clinton', 'R', 'R', 'SEA', 'P', 1989)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'canoj001', 'Cano', 'Joselito', 'R', 'R', 'HOU', 'P', 1989)    
### 1990
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yorkm001', 'York', 'Michael', 'R', 'R', 'PIT', 'P', 1990)
### 1991
## AL    
    rosters_df = add_roster_entry_if_needed(rosters_df, 'coles001', 'Cole', 'Stewart', 'R', 'R', 'KCA', '2B', 1991)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'daltm001', 'Dalton', 'Michael', 'L', 'L', 'DET', 'P', 1991)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zoske001', 'Zosky', 'Edward', 'R', 'R', 'TOR', 'SS', 1991)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zupcb001', 'Zupcic', 'Robert', 'R', 'R', 'BOS', 'RF', 1991)
### 1992
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'woodk001', 'Woodson', 'Walter', 'R', 'R', 'SEA', 'P', 1992)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'colev002', 'Cole', 'Victor', 'B', 'R', 'PIT', 'P', 1992)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'younk001', 'Young', 'Kevin', 'R', 'R', 'PIT', '3B', 1992)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'younp001', 'Young', 'Bryan', 'R', 'R', 'MON', 'P', 1992)   
### 1993
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zambe001', 'Zambrano', 'Eduardo', 'R', 'R', 'CHN', 'RF', 1993)
### 1994
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'youne002', 'Young', 'Ernest', 'R', 'R', 'OAK', 'LF', 1994)
### 1995
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'bradd001', 'Brady', 'Stephen', 'B', 'R', 'CHA', '2B', 1995)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wolcb001', 'Wolcott', 'Robert', 'R', 'R', 'SEA', 'P', 1995)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'courj001', 'Courtright', 'John', 'L', 'L', 'CIN', 'P', 1995)
### 1996
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'duram001', 'Durant', 'Michael', 'R', 'R', 'MIN', 'C', 1996)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wills001', 'Williams', 'Shad', 'R', 'R', 'CAL', 'P', 1996)    
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'bours001', 'Bourgeois', 'Steven', 'R', 'R', 'SFN', 'P', 1996)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wilsd002', 'Wilson', 'Desi', 'L', 'L', 'SFN', '1B', 1996)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zubej001', 'Zuber', 'Jon', 'L', 'L', 'PHI', '1B', 1996)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wilsd002', 'Wilson', 'Desi', 'L', 'L', 'SFN', '1B', 1996)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zubej001', 'Zuber', 'Jon', 'L', 'L', 'PHI', '1B', 1996)
### 1997
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'diaze003', 'Diaz', 'Eddy', 'R', 'R', 'MIL', '2B', 1997)
### 1998
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'conns001', 'Connelly', 'Steven', 'R', 'R', 'OAK', 'C', 1998)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'franm003', 'Frank', 'Stephen', 'L', 'L', 'CIN', 'CF', 1998)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yoshm001', 'Yoshii', 'Masato', 'R', 'R', 'NYN', 'C', 1998)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yount001', 'Young', 'Timothy', 'L', 'L', 'MON', 'P', 1998)
### 1999
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yarne001', 'Yarnall', 'Harvey', 'L', 'L', 'NYA', 'P', 1999)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zimmj001', 'Zimmerman', 'Jeffrey', 'R', 'R', 'TEX', 'P', 1999)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zimmj002', 'Zimmerman', 'Jordan', 'R', 'L', 'SEA', 'P', 1999)
### 2000
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'carpb001', 'Carpenter', 'Charles', 'L', 'L', 'COL', 'C', 2000)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wunsk001', 'Wunsch', 'Kelly', 'L', 'L', 'CHA', 'C', 2000)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zerbc001', 'Zerbe', 'William', 'L', 'L', 'SFN', 'P', 2000)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zulej001', 'Zuleta', 'Julio', 'R', 'R', 'CHN', '1B', 2000)
### 2002
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wrigr002', 'Wright', 'Ronald', 'R', 'R', 'SEA', 'C', 2002)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zinta001', 'Zinter', 'Alan', 'B', 'R', 'HOU', 'C', 2002)
### 2003
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'budzm001', 'Budzinski', 'Mark', 'L', 'L', 'CIN', 'C', 2003)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zoccp001', 'Zoccolillo', 'Peter', 'L', 'R', 'MIL', 'RF', 2003)
### 2005
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'calzn001', 'Calzado', 'Napoleon', 'R', 'R', 'BAL', 'CF', 2005)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'willg003', 'Williams', 'Glenn', 'B', 'R', 'MIN', '3B', 2005)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'younw001', 'Young', 'Walter', 'L', 'R', 'BAL', '1B', 2005)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'chava002', 'Chavez', 'Angel', 'R', 'R', 'SFN', 'SS', 2005)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zimmr001', 'Zimmerman', 'Ryan', 'R', 'R', 'WAS', '3B', 2005)
### 2006
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zumaj001', 'Zumaya', 'Joel', 'R', 'R', 'DET', 'P', 2006)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'youms001', 'Youman', 'Shane', 'L', 'L', 'PIT', 'P', 2006)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yound004', 'Young', 'Delwyn', 'B', 'R', 'LAN', 'RF', 2006)
### 2007
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'cavaa001', 'Cavazos', 'Andres', 'R', 'R', 'SLN', 'C', 2007)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'dejoj001', 'De Jong', 'Jordan', 'R', 'R', 'TOR', 'P', 2007)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wrigc001', 'Wright', 'Sebern', 'L', 'L', 'NYA', 'P', 2007)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zaram001', 'Zarate', 'Mauro', 'R', 'R', 'FLO', 'P', 2007)
### 2008
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yabuy001', 'Yabuta', 'Yasuhiko', 'R', 'R', 'KCA', 'P', 2008)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zinkc001', 'Zink', 'Charles', 'R', 'R', 'BOS', 'P', 2008)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'worrm001', 'Worrell', 'Mark', 'R', 'R', 'SLN', 'P', 2008)
### 2009
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'canib001', 'Canizares', 'Barbaro', 'R', 'R', 'ATL', 'C', 2009)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'burkg001', 'Burke', 'Gregory', 'R', 'R', 'SDN', 'C', 2009)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'david004', 'Davidson', 'Daniel', 'L', 'L', 'ANA', 'P', 2009)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'woodt003', 'Wood', 'Timothy', 'R', 'R', 'FLO', 'C', 2009)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zavac002', 'Zavada', 'Clay', 'L', 'L', 'ARI', 'C', 2009)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'youne003', 'Young', 'Eric', 'B', 'R', 'COL', 'CF', 2009)
### 2010
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'curtc002', 'Curtis', 'Colin', 'L', 'L', 'NYA', 'LF', 2010)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'cedaj001', 'Ceda', 'Jose', 'R', 'R', 'FLO', 'P', 2010)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zawal001', 'Zawadzki', 'Lance', 'B', 'R', 'SDN', '2B', 2010)
### 2011
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'younm004', 'Young', 'Matthew', 'L', 'R', 'ATL', 'CF', 2011)
### 2012
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'verdr001', 'Verdugo', 'Ryan', 'L', 'L', 'KCA', 'P', 2012)
### 2013
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wootr001', 'Wooten', 'Rob', 'R', 'R', 'MIL', 'C', 2013)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zeidj001', 'Zeid', 'Joshua', 'R', 'R', 'HOU', 'P', 2013)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'burgh001', 'Burgos', 'Hiram', 'R', 'R', 'MIL', 'P', 2013)
### 2014
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'darnl001', 'Darnell', 'Logan', 'L', 'L', 'MIN', 'P', 2014)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'browg001', 'Brown', 'Gary', 'R', 'R', 'SFN', 'CF', 2014)
### 2015
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'deckc001', 'Decker', 'Cody', 'R', 'R', 'SDN', '1B', 2015)
### 2016
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'coatj001', 'Coats', 'Jason', 'R', 'R', 'CHA', 'RF', 2016)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'cabrm003', 'Cabrera', 'Mauricio', 'R', 'R', 'ATL', 'C', 2016)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wimma001', 'Wimmers', 'Alex', 'L', 'R', 'MIN', 'P', 2016)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'ynoam001', 'Ynoa', 'Michael', 'R', 'R', 'CHA', 'P', 2016)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'buscm002', 'Buschmann', 'Matthew', 'R', 'R', 'ARI', 'P', 2016)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'cabrm003', 'Cabrera', 'Mauricio', 'R', 'R', 'ATL', 'P', 2016)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'ynoag001', 'Ynoa', 'Gabriel', 'R', 'R', 'NYN', 'P', 2016)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'ynoam001', 'Ynoa', 'Michael', 'R', 'R', 'CHA', 'C', 2016)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'younm005', 'Younginer', 'John', 'R', 'R', 'ATL', 'P', 2016)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'buscm002', 'Buschmann', 'Matthew', 'R', 'R', 'ARI', 'P', 2016)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'ynoag001', 'Ynoa', 'Gabriel', 'R', 'R', 'NYN', 'P', 2016)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'younm005', 'Younginer', 'John', 'R', 'R', 'ATL', 'P', 2016)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wilka002', 'Wilkins', 'Andrew', 'L', 'R', 'MIL', 'RF', 2016)
### 2017
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zimmb001', 'Zimmer', 'Bradley', 'L', 'R', 'CLE', 'CF', 2017)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wimma001', 'Wimmers', 'Alex', 'L', 'R', 'MIN', 'C', 2017)
### 2018
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'cuevn001', 'Cuevas', 'Noel', 'R', 'R', 'COL', 'C', 2018)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zamod001', 'Zamora', 'Daniel', 'L', 'L', 'NYN', 'C', 2018)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zamod001', 'Zamora', 'Daniel', 'L', 'L', 'NYN', 'P', 2018)
### 2019
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yastm001', 'Yastrzemski', 'Michael', 'L', 'L', 'SFN', 'C', 2019)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zavas001', 'Zavala', 'Bernardo', 'R', 'R', 'CHA', 'C', 2019)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zeuct001', 'Zeuch', 'Timothy', 'R', 'R', 'TOR', 'P', 2019)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yamaj001', 'Yamamoto', 'Jordan', 'R', 'R', 'MIA', 'P', 2019)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zeuct001', 'Zeuch', 'Timothy', 'R', 'R', 'TOR', 'C', 2019)
### 2020
## AL    
    rosters_df = add_roster_entry_if_needed(rosters_df, 'woodj003', 'Woodford', 'Jacob', 'R', 'R', 'SLN', 'C', 2020)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yajum001', 'Yajure', 'Miguel', 'R', 'R', 'NYA', 'P', 2020)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zimmb002', 'Zimmermann', 'Bruce', 'L', 'L', 'BAL', 'C', 2020)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zubet001', 'Zuber', 'John', 'R', 'R', 'KCA', 'P', 2020)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'youna003', 'Young', 'Andrew', 'R', 'R', 'ARI', '3B', 2020)
### 2021
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'brujv001', 'Brujan', 'Vidal', 'B', 'R', 'TBA', '2B', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'cleme002', 'Clement', 'Ernie', 'R', 'R', 'CLE', '2B', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'willl001', 'Williams', 'Lucas', 'R', 'R', 'PHI', 'C', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wilsj005', 'Wilson', 'Jacob', 'R', 'R', 'HOU', '3B', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wilsj005', 'Wilson', 'Jacob', 'R', 'R', 'OAK', '2B', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'benda001', 'Bender', 'Anthony', 'R', 'R', 'MIA', 'C', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'carld003', 'Carlton', 'Drew', 'R', 'R', 'DET', 'P', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'coled003', 'Coleman', 'Dylan', 'R', 'R', 'KCA', 'P', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yangh001', 'Yang', 'Hyeon-Jong', 'L', 'L', 'TEX', 'P', 2021)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'benda001', 'Bender', 'Anthony', 'R', 'R', 'MIA', 'P', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'bosta001', 'Bostick', 'Akeem', 'R', 'R', 'NYN', 'P', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'bruij001', 'Bruihl', 'Justin', 'L', 'L', 'LAN', 'P', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'carld003', 'Carlton', 'Drew', 'R', 'R', 'DET', 'C', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'welkc001', 'Welker', 'Colton', 'R', 'R', 'COL', '3B', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yajum001', 'Yajure', 'Miguel', 'R', 'R', 'PIT', 'P', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yangh001', 'Yang', 'Hyeon-Jong', 'L', 'L', 'TEX', 'C', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zimmb002', 'Zimmermann', 'Bruce', 'L', 'L', 'BAL', 'C', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zubet001', 'Zuber', 'John', 'R', 'R', 'KCA', 'C', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'bosta001', 'Bostick', 'Akeem', 'R', 'R', 'NYN', 'P', 2021)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'bruij001', 'Bruihl', 'Justin', 'L', 'L', 'LAN', 'P', 2021)
### 2022
## AL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yepej001', 'Yepez', 'Juan', 'R', 'R', 'SLN', 'C', 2022)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'weisg001', 'Weissert', 'Gregory', 'R', 'R', 'NYA', 'P', 2022)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wentj001', 'Wentz', 'Joseph', 'L', 'L', 'DET', 'P', 2022)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wilec001', 'Wiles', 'Collin', 'R', 'R', 'OAK', 'P', 2022)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'wilss002', 'Wilson', 'Steven', 'R', 'R', 'SDN', 'C', 2022)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'woods002', 'Woods Richardson', 'Simeon', 'R', 'R', 'MIN', 'C', 2022)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'yound005', 'Young', 'Daniel', 'L', 'L', 'SEA', 'P', 2022)
    rosters_df = add_roster_entry_if_needed(rosters_df, 'zabaa001', 'Zabala', 'Aneurys', 'R', 'R', 'MIA', 'C', 2022)
## NL
    rosters_df = add_roster_entry_if_needed(rosters_df, 'younj003', 'Young', 'Jared', 'L', 'R', 'CHN', '1B', 2022)

    return rosters_df

def fix_special_case_hands(events_df):
# 1914
## FL
    events_df.loc[events_df['batterID'] == 'tappw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wille103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wistt101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yerks101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yound104', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zinng101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zwild101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wille103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yound104', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zinng101', 'pitcher_hand'] = 'R'
## AL
    events_df.loc[events_df['batterID'] == 'willg103', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willh103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willl104', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wolfp101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woodr103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wyckj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zeism101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willl104', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wolfp101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodr103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wyckj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zeism101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'tincb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wagnh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wheaz101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'whitp102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilth102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wingi101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'yinge101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zimmh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'tincb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilth102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'yinge101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zimmh101', 'pitcher_hand'] = 'R'
# 1915
## FL
    events_df.loc[events_df['batterID'] == 'camnh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dougl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'downt101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'drakd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'enzej101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'falkc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'farrj104', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'kaisg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'camnh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dougl101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'falkc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'kaisg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stovg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitg103', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'wrigd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zwild101', 'pitcher_hand'] = 'L'
## AL 
    events_df.loc[events_df['batterID'] == 'colek101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colle101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'connt101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'conwo101', 'batter_hand'] = '?'
    events_df.loc[events_df['batterID'] == 'coumf101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coveh101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'gaind101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'colek101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coumf101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'coveh101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'warhj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willr104', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wolfm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodj108', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'younr103', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'warhj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willr104', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wolfm101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodj108', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'chalg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coopw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crutd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'daleg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'darir101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'daubj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'doyll101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fluhj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chalg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coopw101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'crutd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'daleg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wendl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whalb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsc102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zimmb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wagnh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whalb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whitp102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wingi101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zimmb101', 'pitcher_hand'] = 'R'    
###1916
## AL
    events_df.loc[events_df['batterID'] == 'craws101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dealc102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dumog101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'felsh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gregv101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dumog101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'waltr102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'weilc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wrigc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'weilc101', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'connj107', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'corhr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dellw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fleta101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grift102', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dellw101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilhj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilsa102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilhj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsa102', 'pitcher_hand'] = 'R'
### 1917
## AL
    events_df.loc[events_df['batterID'] == 'culln101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cunng101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davih101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'debeh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gardl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gharp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hamie101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'culln101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cunng101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hamie101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'waldd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'warda101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wittw101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wrigr104', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'waldd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wittw101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigr104', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'carlh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cravg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dillp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'doakb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'everj102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fabrb101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'hemie101', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'carlh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'doakb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'watsm102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wolfh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zeidr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'watsm102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wheaz101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zeidr101', 'pitcher_hand'] = 'R'    
### 1918
## AL
    events_df.loc[events_df['batterID'] == 'casej101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'enzmj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'faber101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'finnh102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hendt101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'enzmj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'faber101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'finnh102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zacht101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'warda101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zacht101', 'pitcher_hand'] = 'L'    
## NL
    events_df.loc[events_df['batterID'] == 'brott101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'catoh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crumc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dougp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'elleh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fitzm102', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'crumc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dougp101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'elleh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stenc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wheed102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodf102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'stenc101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'woodf102', 'pitcher_hand'] = 'R'        
### 1919
## AL
    events_df.loc[events_df['batterID'] == 'davii102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dugaj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'evanj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gallb103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gandc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hoytw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kanef101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'gallb103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hoytw101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wicka101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zinnj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wicka101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'winng101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zinnj101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'cantm101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chash101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cruiw101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fishs102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'flacm101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gerne101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cantm101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'gerne101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'weavh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yeabb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zitzb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'weavh101', 'pitcher_hand'] = 'R'
### 1920
## AL
    events_df.loc[events_df['batterID'] == 'conwj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'debej101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fortg101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'grifi101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hodgs101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hofmf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'conwj101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'debej101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ellig101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fortg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hodgs101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willk101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'willk101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'davec101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fordh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'griet102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kildp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'davec101', 'pitcher_hand'] = 'R'
### 1921
## AL
    events_df.loc[events_df['batterID'] == 'cobbt101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'erice101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'freeh102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'granj104', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'harph101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'erice101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'freeh102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'harph101', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'connj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'donop102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'filld101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gilhg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'janvh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'keenj103', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'donop102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'filld101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'keenj103', 'pitcher_hand'] = 'L'
### 1922
## AL
    events_df.loc[events_df['batterID'] == 'dursc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ehmkh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fewsc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gallc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'jones104', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cox-e101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ehmkh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'jones104', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yaryy101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'waltr102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodl101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yaryy101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younr103', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'cunnb102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'duncp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eubau101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grimb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kingl103', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'eubau101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'grimb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'walkr102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wirtk101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigr102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'younr101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'walkr102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigr102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younr101', 'pitcher_hand'] = 'R'    
### 1923
## AL
    events_df.loc[events_df['batterID'] == 'collr104', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'conns101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'edmog101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hastb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'collr104', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'conns101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'edmog101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hastb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'winng101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woodd104', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zahnp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zahnp101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'carem101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'collp102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cousd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dickl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'doutt101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gasta101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'glazw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'collp102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dickl101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'glazw101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'trayp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'watsm103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wigif101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'watsm103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wigif101', 'pitcher_hand'] = 'R'    
### 1924
## AL
    events_df.loc[events_df['batterID'] == 'daush101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'david102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'drakl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'flagi101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gehrl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gibsc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hargp101', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'daush101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'david102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'drakl101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'welcf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'winge102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'workh102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yowec101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'welcf101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'winge102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'workh102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yowec101', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'churj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cloue101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'crith101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ens-j101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fourj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'frisf101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'vineb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wheer101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yarrr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yde-e101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'yeara101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'vineb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wheer101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yarrr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yde-e101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'yeara101', 'pitcher_hand'] = 'R'
### 1925
## AL
    events_df.loc[events_df['batterID'] == 'englc102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ezzeh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ferga101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ferga101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gregv101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'thurs101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willl103', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'thurs101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willl103', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'bushg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'decaa101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'farrd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bushg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'decaa101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'warwb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'weisb102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilsf101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'weisb102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wilsf101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zitzb101', 'pitcher_hand'] = 'R'
### 1926
## AL
    events_df.loc[events_df['batterID'] == 'combe101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'croub101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'falkc102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'foreh101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'foxxj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hadlb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'jamic101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'falkc102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'foreh101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'hadlb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'uchrj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'uhleg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'veltp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'uchrj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'uhleg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'veltp101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'comoa101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coonj103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cummj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dyere101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'genej101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dyere101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'genej101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsh102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yarnr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsh102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yarnr101', 'pitcher_hand'] = 'R'    
### 1927
## AL
    events_df.loc[events_df['batterID'] == 'cochm101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coves101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dixol101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eichi101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fothb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'browj102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coves101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'clarb102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cuylk101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deitb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dresc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clarb102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ussad101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'weraj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willk102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wingt101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yerkc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'weraj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willk102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wingt101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yerkc101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'topos101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'vancd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'walkb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wertj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yotee101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'topos101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'vancd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'walkb101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wertj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yotee101', 'pitcher_hand'] = 'R'    
### 1928
## AL
    events_df.loc[events_df['batterID'] == 'burkb102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'burng102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clanb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'crowg102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dykej101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grifm102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burkb102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'crowg102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'grifm102', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'burwb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cohea101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ehrhr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gonzm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'greej103', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hafec101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burwb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ehrhr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'greej103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsr101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilth101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wrigj104', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsr101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wilth101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wrigj104', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'westm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wille102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsj105', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigg102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'westm101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigg102', 'pitcher_hand'] = 'R'
### 1929
## AL
    events_df.loc[events_df['batterID'] == 'carre101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cissb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cronj103', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'grahs101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grang101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carre101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'grahs101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'grang101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stree101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'weilb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'stree101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'weilb101', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'clarw101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'collp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cvenm101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dawsj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'franf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clarw101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cvenm101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dawsj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'franf101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'welsj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'westa101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willc103', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'windb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'welsj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'westa101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willc103', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wilsj105', 'pitcher_hand'] = 'R'        
### 1930
## AL
    events_df.loc[events_df['batterID'] == 'carap101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chilh101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'dickb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'eastp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'falkb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'carap101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'chilh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'storl101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'wests101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willd107', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willh102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'winer101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'winst101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wyatw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willd107', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wyatw101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'coonj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'englw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fishs101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fitzf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fribb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coonj101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'faulj101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'fitzf101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woods101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'woods101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wysob101', 'pitcher_hand'] = 'L'    
### 1931
## AL
    events_df.loc[events_df['batterID'] == 'coonb102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'finnl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gomel102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'liseh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coonb102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gomel102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'liseh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vosmj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'welle101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'younr102', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'vosmj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'welle101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'winst101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'cronb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'derrp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'flowj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fullc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'derrp101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wysob101', 'batter_hand'] = 'L'
### 1932
## AL
    events_df.loc[events_df['batterID'] == 'campb102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'connb104', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cramd102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'crosf103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'durhe101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'edele101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ferrr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'haywr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'durhe101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'edele101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'welcj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wisea101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'storl101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'welcj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wests101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'winer101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wisea101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'cunnb104', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deand102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'finnh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fredj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'freyb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grimc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'healf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cunnb104', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'deand102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'freyb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wanep101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'winfj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willh102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'winfj101', 'pitcher_hand'] = 'R'    
### 1933
## AL
    events_df.loc[events_df['batterID'] == 'chape101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'coomb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cragh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fonsl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'frasv101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chape101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coomb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cragh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'frasv101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'werbb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'white102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zapuj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'werbb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'white102', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'cucct101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'delke101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'durol101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grace101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hubbc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hubbc101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'stoua101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'weinp101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'younp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'stoua101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'weinp101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'younp101', 'pitcher_hand'] = 'R'
### 1934
## AL
    events_df.loc[events_df['batterID'] == 'carsk101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coffd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cohes101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'conlj102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'devec101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dietb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fox-p102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grovl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'coffd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cohes101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'devec101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dietb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'grovl101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'walkd101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilsw103', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'yorkr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'walkd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsw103', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'chapg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'darrg101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'delab102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'grang102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hacks101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'darrg101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wiedc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wistw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wortr102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wiedc101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wistw101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wortr102', 'pitcher_hand'] = 'R'    
### 1935
## AL
    events_df.loc[events_df['batterID'] == 'calde101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'conrb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dahlb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gehrc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'grubf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hayej103', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'calde101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hayej103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitj104', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilsj103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wriga101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whitj104', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsj103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wriga101', 'pitcher_hand'] = 'R'        
## NL
    events_df.loc[events_df['batterID'] == 'chiol101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coscj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'frenl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'frenl101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'sullb103', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'warnl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitp103', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'sullb103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'warnl101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whitp103', 'pitcher_hand'] = 'R'    
### 1936
## AL
    events_df.loc[events_df['batterID'] == 'caret102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'doylc102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'evanr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fellb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'glenj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'doylc102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'evanr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fellb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wickk101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zubeb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wickk101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zubeb101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'chapt101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dickj102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'freit101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gablf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burke102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chapt101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'freit101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'gablf101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'walkh101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'younb102', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'walkh101', 'pitcher_hand'] = 'R'    
### 1937
## AL
    events_df.loc[events_df['batterID'] == 'bottj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chapb102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cox-b102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deanc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cox-b102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'deanc101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'weavm101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'whitj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'youne101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'weavm101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whitj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yorkr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'youne101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'camid102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cherp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crafh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'deanp101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stait101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'weirr101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'whitb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yound105', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'stait101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'weirr101', 'pitcher_hand'] = 'L'
### 1938
## AL
    events_df.loc[events_df['batterID'] == 'chaps101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'daviw101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dicke101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'donaa101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'haleo101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'daviw101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dicke101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'donaa101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willa101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigt102', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'willa101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigt102', 'pitcher_hand'] = 'R'        
## NL
    events_df.loc[events_df['batterID'] == 'corbg101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'davic103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davik101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eppea101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'errid101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'davic103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'eppea101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'errid101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tripc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'westd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'westm102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wittj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'tripc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wittj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yound105', 'pitcher_hand'] = 'R'        
### 1939
## AL
    events_df.loc[events_df['batterID'] == 'chask101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'colle102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cullr101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'desag101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'drakt101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ferrw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chask101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'drakt101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ferrw101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'sunds101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wagnh102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wynne101', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'sunds101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wynne101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'brunr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'butcm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cresc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dimav101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'earlt101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gleej101', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'brunr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'butcm101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'earlt101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wietw101', 'batter_hand'] = 'B'    
### 1940
## AL
    events_df.loc[events_df['batterID'] == 'dimad101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dimaj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'earlj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gallj103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hayej101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dorsc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wallj102', 'batter_hand'] = 'B'    
## NL
    events_df.loc[events_df['batterID'] == 'browl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'davis101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gumbh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'browl101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'gumbh101', 'pitcher_hand'] = 'R'    
    events_df.loc[events_df['batterID'] == 'warsr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'white103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willa103', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'white103', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'willa103', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'younb102', 'pitcher_hand'] = 'L'        
### 1941
## AL
    events_df.loc[events_df['batterID'] == 'franm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hajdc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'trotb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'washg103', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wolfr102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'workc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'trotb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'washg103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wolfr102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'workc101', 'pitcher_hand'] = 'R'        
## NL
    events_df.loc[events_df['batterID'] == 'browj110', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clemb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'croub103', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'demaf102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dusae101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'galaa101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'goodi101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'browj110', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clemb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'croub103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'waite101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'warrb102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilkl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zienb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'warrb102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilkl101', 'pitcher_hand'] = 'L'
### 1942
## AL
    events_df.loc[events_df['batterID'] == 'chrir101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clare102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crist101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gorsj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'heimv101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'chrir101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gorsj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'turnj102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'weatr101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willt103', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'yankg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'turnj102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'weatr101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'coscp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eavev101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'feldh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'flith101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'eavev101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'feldh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'flith101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wyroj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wyseh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'westd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wyseh101', 'pitcher_hand'] = 'R'    
### 1943
## AL
    events_df.loc[events_df['batterID'] == 'doerb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grovo101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'higgm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'grovo101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'webbs101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodg101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woodp102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zaria101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'webbs101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodp102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zaria101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'cuccc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dagej101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'goldl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cuccc101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dagej101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'webbb102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'webbb102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willw101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wyroj101', 'pitcher_hand'] = 'R'    
### 1944
## AL
    events_df.loc[events_df['batterID'] == 'curtv101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'derrr101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'embrr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eppsh101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'curtv101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'embrr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'suscg102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilkb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zolds101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wilkb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zinsb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zolds101', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'branr103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'delat102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'donad101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eastr102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'etchb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fallg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'filis101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'branr103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'delat102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'donad101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wasdj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilkt101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yorkt102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zachc102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zak-f101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wasdj101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wilkt101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zachc102', 'pitcher_hand'] = 'R'
### 1945
## AL
    events_df.loc[events_df['batterID'] == 'cronj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'evana101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grayp101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'greeh102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tresm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willp102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zardj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willp102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zardj101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'byerb103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'drewf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'flagw101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'byerb103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chapb102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitb106', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrige101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zimmr101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'whitb106', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrige101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zimmr101', 'pitcher_hand'] = 'L'
### 1946
## AL
    events_df.loc[events_df['batterID'] == 'colmf101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'conwj103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deutm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fannc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gentr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gracj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'deutm101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fannc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gentr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whith101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yoste101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whith101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yoste101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'budnm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cameh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chipb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dickm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fernn101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gilbc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'budnm101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chipb101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dassf101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dickm101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitd103', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'whitd103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zienb101', 'pitcher_hand'] = 'R'
### 1947
## AL
    events_df.loc[events_df['batterID'] == 'dorsc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gillb102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'houta101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gillb102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'houta101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'valoe101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wighb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'valoe101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wighb101', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'caseh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coopm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cox-b101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ellig102', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'freyl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'haasb102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'caseh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coopm101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ellig102', 'pitcher_hand'] = 'L'
### 1948
## AL
    events_df.loc[events_df['batterID'] == 'dobyl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'drakl102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dreic101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'drewk101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ferrd101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ginsj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dreic101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'drewk101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ferrd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stirs101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigt101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'walkh102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'walkr101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilsg103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yvars101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'walkh102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'walkr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsg103', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'culld101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ellib103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eraue101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ericp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gregh102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'eraue101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ericp101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gregh102', 'pitcher_hand'] = 'R'
### 1949
## AL
    events_df.loc[events_df['batterID'] == 'cainb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'collj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fainf101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hutcf101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cainb101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'hutcf101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zerng101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zerng101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'chamc102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dobej101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ennid101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chamc102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cresw101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dobej101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'voisb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'westw102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zabaa101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'voisb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'westw102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yochr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zabaa101', 'pitcher_hand'] = 'L'
### 1950
## AL
    events_df.loc[events_df['batterID'] == 'carrc105', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clara102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'delsj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dorif101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grasm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grayt101', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'dorif101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodk101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'workh101', 'batter_hand'] = 'L'    
    events_df.loc[events_df['pitcherID'] == 'woodk101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'campr102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chesb101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'churb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coopw102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crand102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dealc101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'hearj102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'jackr102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chesb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'churb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dealc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hearj102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vancc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'waltb102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wardp101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'vancc101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'waltb102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wardp101', 'pitcher_hand'] = 'R'    
### 1951
## AL
    events_df.loc[events_df['batterID'] == 'coler102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'evanb103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fedea101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'evanb103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'younb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zaucn101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zuveg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zuveg101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'clarm102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dempc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'donod101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dubim101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'edwah101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dempc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'donod101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dubim101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willd104', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsj104', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yochl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'yound101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'willd104', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsj104', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yochl101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'yvars101', 'pitcher_hand'] = 'R'    
### 1952
## AL
    events_df.loc[events_df['batterID'] == 'cervb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deloi101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fornm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fricm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'deloi101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fornm101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fricm101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'clarm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davib103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'erskc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clarm101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'erskc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'usheb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wolfe101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yuhae101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'usheb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wolfe101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yound101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yuhae101', 'pitcher_hand'] = 'R'    
### 1953
## AL
    events_df.loc[events_df['batterID'] == 'carsf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'conss101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davay101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eastl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fanof101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'flowb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'conss101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fanof101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'flowb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigt101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'bridr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cavap101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coled103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'furic101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hansa101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coled103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hansa101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'waugj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wehmh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willj104', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'waugj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wehmh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willj104', 'pitcher_hand'] = 'R'    
### 1954
## AL
    events_df.loc[events_df['batterID'] == 'bullg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carea101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colej101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dobsj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'finij101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garcm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'korcs101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'colej101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dobsj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'garcm101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsr102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younb101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'collj102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gardb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hallb105', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'collj102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'worta101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zickb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zimmd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'worta101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zickb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zimmd101', 'pitcher_hand'] = 'R'    
### 1955
## AL
    events_df.loc[events_df['batterID'] == 'burtm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chakb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dalep101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'delij101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'everh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burtm101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chakb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'white106', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wiesb101', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'white106', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wiesb101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zaucn101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'commj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cronr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'frazj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'freeg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cronr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'valec101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wadeg101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'watef101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'westj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'whitc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willd102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'valec101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'watef101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'whitc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willd102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigm101', 'pitcher_hand'] = 'R'    
### 1956
## AL
    events_df.loc[events_df['batterID'] == 'bridj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cecca101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colej106', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'daleb102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fitze101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cecca101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'daleb102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilsg101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woodh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodh101', 'pitcher_hand'] = 'L'    
## NL
    events_df.loc[events_df['batterID'] == 'burgs101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coviw101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cunnj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'draks101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'foilh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gomer101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'darnb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gomer101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'thurb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'trowb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wince101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'thurb101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'trowb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wince101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigr101', 'pitcher_hand'] = 'R'
### 1957
## AL
    events_df.loc[events_df['batterID'] == 'cox-g101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'espos101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cox-g101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'turlb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yewct101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'turlb101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'darka101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dottd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'farrt101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'elstd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'farrt101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'smalr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whisp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilhh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'smalr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whisp101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilhh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wittr101', 'pitcher_hand'] = 'R'
### 1958
## AL
    events_df.loc[events_df['batterID'] == 'bunnj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'casaj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colar101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dropw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'durer101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bunnj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'casaj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'colar101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'durer101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsd101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wilsd101', 'pitcher_hand'] = 'L'    
## NL
    events_df.loc[events_df['batterID'] == 'clarp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cokej101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'consj101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'drysd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'freeg102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clarp101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'consj101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'drysd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willb106', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wisec101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'wittr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zannd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willb106', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wisec101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zannd101', 'pitcher_hand'] = 'R'
### 1959
## AL
    events_df.loc[events_df['batterID'] == 'burnp102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chitn101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dobbd101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'burnp102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'chitn101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coler101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'websr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilse102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'windg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'websr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilse102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'windg101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'carmd101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'conlg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cottc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cuelm101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'davej101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'conlg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cuelm101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'thomv101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitb105', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willb104', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'whitb105', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'willb104', 'pitcher_hand'] = 'R'    

    
### 1960
## AL
    events_df.loc[events_df['batterID'] == 'browh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chith101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'consb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'delam101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'demaj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ferrd102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'browh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ferrd102', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'brewj102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'burte101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cardl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cepeo101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clemr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'brewj102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wieat101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willc104', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wills102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsn101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woodj103', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wieat101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willc104', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wills102', 'pitcher_hand'] = 'R'
### 1961
## AL
    events_df.loc[events_df['batterID'] == 'causw101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'earla101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'earla101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woodj107', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yastc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zipfb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zupof101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wyatj102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zipfb101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'brese101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brosj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'burdl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clemd101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'facer101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fairr101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'brosj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burdl101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'facer101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zimmj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodj103', 'pitcher_hand'] = 'R'    
### 1962
## AL
    events_df.loc[events_df['batterID'] == 'chand102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cimog101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'danib102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'debud101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'donoj103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fishj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chand102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'danib102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'debud101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'donoj103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fishj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'weavf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wyatj102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'weavf101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodj107', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'bollf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brocl102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'busbj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chrij101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dulib101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'edwaj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'frane101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gillj102', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'dulib101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'frane101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'sanfj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'virdb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'whitf103', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wineb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'sanfj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whitf103', 'pitcher_hand'] = 'L'    
### 1963
## AL
    events_df.loc[events_df['batterID'] == 'browl104', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bunkw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dottg101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dustb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fregj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'geigg101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'bunkw101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dottg101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dustb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'valef101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'wagnl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'valef101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wagnl101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'callj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cardc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cardj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ciscg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clend101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clint101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ferra101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cardc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ciscg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stana101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'withc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zachc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'stana101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wineb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'withc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodw101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zachc101', 'pitcher_hand'] = 'R'    
### 1964
## AL
    events_df.loc[events_df['batterID'] == 'boswd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'browd102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chanb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gibbj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'grayd103', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'boswd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fishf101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'grayd103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'torrf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vined101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'torrf101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'vined101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yastc101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'brucb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'burdf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crair101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'duffj102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ellis101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'brucb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burdf101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'crair101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'duffj102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ellis101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'snidd101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willn101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wiser101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodw103', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'snidd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willn101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wiser101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodw103', 'pitcher_hand'] = 'L'    
### 1965
## AL
    events_df.loc[events_df['batterID'] == 'blefc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'clinl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deesc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fishe102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'greel101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hallj107', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hamij101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fishe102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hamij101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wantd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'warwc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitr101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'willt102', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wantd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'warwc101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'whitr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willt102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zimmj101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'bertd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'burdb102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cowab101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'flooc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'spahw101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'yound103', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'spahw101', 'pitcher_hand'] = 'L'
### 1966
## AL
    events_df.loc[events_df['batterID'] == 'buzhj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'casap101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'egand101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'foy-j101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gladf102', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'buzhj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'egand101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'gladf102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'watte101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'womad101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wrigc102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'watte101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'womad101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigc102', 'pitcher_hand'] = 'L'    
## NL
    events_df.loc[events_df['batterID'] == 'browb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carrc103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colbn101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cosmj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dietd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carrc103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cosmj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'terrr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'virgo101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'walkl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'terrr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'virgo101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'walkl101', 'pitcher_hand'] = 'L'    
### 1967
## AL
    events_df.loc[events_df['batterID'] == 'bowes103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'conit101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'conne102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dobsc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'downa101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'granj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'conne102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dobsc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'downa101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'granj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'websr102', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'websr102', 'pitcher_hand'] = 'L'    
## NL
    events_df.loc[events_df['batterID'] == 'branr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'calmd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chare101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coleg101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'deant101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'frymw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gaglp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'calmd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'frymw101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'sorrb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'werhj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willm102', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'wynnb102', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'sorrb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'werhj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willm102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wynnb102', 'pitcher_hand'] = 'R'    
### 1968
## AL
    events_df.loc[events_df['batterID'] == 'cumbj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davav101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'duncd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'holtj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cumbj101', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'coltl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'connb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'corrp102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dalcb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fastd101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'coltl101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'connb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dalcb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fastd101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'schrb102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'versz101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'weisa101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'willr102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wynnj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'schrb102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'weisa101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willr102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wynnj101', 'pitcher_hand'] = 'R'    
### 1969
## AL
    events_df.loc[events_df['batterID'] == 'branb102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brinc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brung101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cox-b103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cramg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cridj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ellsd101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hillj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'branb102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'brung101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cramg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cridj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ellsd101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'hillj101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woodr102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zeppb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'versz101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodr102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zeppb101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'britj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clont101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'collk101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'davaj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davit103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deckj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dyerd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'britj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clont101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'deckj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willj108', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willj108', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yound103', 'pitcher_hand'] = 'R'    
### 1970
## AL
    events_df.loc[events_df['batterID'] == 'carer001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'colsl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'comew101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dalrc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'donaj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'doylp101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'drabm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'colsl101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'curtj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'doylp101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'drabm101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'uhlat101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'unsed101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wickf101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'yorkj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'uhlat101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'unsed101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wickf101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yorkj101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'camps102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cardd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chand101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cookr101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'doyld102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fairj102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'folkr101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'garvs001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'camps102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cardd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cookr101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'folkr101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willb105', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wille101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willb105', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wille101', 'pitcher_hand'] = 'R'    
### 1971
## AL
    events_df.loc[events_df['batterID'] == 'burbb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chamc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'clarr103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colej102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'curtj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fitza101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'gelnj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'herre102', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'burbb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'colej102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fitza101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gelnj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wertd103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yatea101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wertd103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yatea101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'cannc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carls001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'culvg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'daviw102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dufff101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'folit001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carls001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'culvg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willj102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ziskr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willj102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younl101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ziskr101', 'pitcher_hand'] = 'R'    
### 1972
## AL
    events_df.loc[events_df['batterID'] == 'bradt102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coggr101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'colbj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colec102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'culpr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'darwb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dukej101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'floyb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'petef101', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'bradt102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'colbj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'culpr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dukej101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'suark101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vossb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'waslg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilcm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodd102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigk101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yanch101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'suark101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'vossb101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'waslg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilcm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodd102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigk101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'carbb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chamb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ellid101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'frisd101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fuent101', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'chamb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ellid101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'frisd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'thomd001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'webbh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yeags001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'thomd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'webbh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsb101', 'pitcher_hand'] = 'R'    
### 1973
## AL
    events_df.loc[events_df['batterID'] == 'bradb105', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'burrj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cabee001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cartr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clarh101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'epstm101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'evand002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'felsj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'campb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dragd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'farme101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'trilm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'veryt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vukoj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willw103', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'waitr001', 'pitcher_hand'] = 'L'    
## NL
    events_df.loc[events_df['batterID'] == 'campd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cox-j102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crawj102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dacqj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davib101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'devia101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'crawj102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dacqj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'devia101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tomld001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'velaf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'winfd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zahng001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'tomld001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'velaf101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'winfd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zahng001', 'pitcher_hand'] = 'L'    
### 1974
## AL
    events_df.loc[events_df['batterID'] == 'dempr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bosmd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'butlb101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'clevr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tovac101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'valeb102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'washh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wockj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wohlj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'younr001', 'batter_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'bryar101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'chril101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'corrv101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cromw101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dennj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dilom001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'easlm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'espin101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fraik101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'bryar101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'chril101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dennj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'espin101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fraik101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'rosep001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'thomm102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tysom101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zamoo101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'thomm102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zamoo101', 'pitcher_hand'] = 'R'
### 1975
## AL
    events_df.loc[events_df['batterID'] == 'brohj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bryes101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chanc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colld001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cubbm101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'freeb103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'healf102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'brobp101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gogob101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'torrm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilem101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'clarj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'concd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cruzh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cruzj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dejei001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garra101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'garrw101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hebnr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'undet001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wallm102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wernd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'undet001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wallm102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wernd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whitj001', 'pitcher_hand'] = 'R'    
### 1976
## AL
    events_df.loc[events_df['batterID'] == 'ceror001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dadep101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dauer001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'briln101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burgt001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'grils101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stilr102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'waitr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woodg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wyneb001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'umbaj101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wheeg101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'cosgm101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'darcp101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dettt101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dresr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fosta101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cosgm101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'darcp101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dettt101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dresr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fosta101', 'pitcher_hand'] = 'R'
### 1977
## AL
    events_df.loc[events_df['batterID'] == 'brigd101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'colub101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'craww101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cruzj002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'decid001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'heisb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'blylb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clanj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cortb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'figue101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'foucs101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willb101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'willm105', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zdebj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zebeg101', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'stepe101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'walkt103', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'caldm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'campr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cling101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'evand001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'forst001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'freid101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grotj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'caldm001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'campr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'forst001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'freid101', 'pitcher_hand'] = 'R'    
    events_df.loc[events_df['batterID'] == 'wallj101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'wartd101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'watsb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whisl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'white001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'younj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zachp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wallj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wartd101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'watsb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whisl101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'white001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yeags001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zachp001', 'pitcher_hand'] = 'R'    
    
### 1978
## AL
    events_df.loc[events_df['batterID'] == 'breaj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chald101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cox-t101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cripd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'doylb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'etcha101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fordd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burks102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fidrm101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodd103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yuraj101', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'stodt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'tanaf001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'toddj102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wortr101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'younk101', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'cannj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cruzt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dwyej001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ferrs101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'forsb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'friap101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garmm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'smitl002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vuckp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'smitl002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'vuckp001', 'pitcher_hand'] = 'R'
### 1979
## AL
    events_df.loc[events_df['batterID'] == 'campb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colbm101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crair001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'david101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grosw001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'clayk101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'comes001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ericr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'thond001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wooda001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'stons101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'vasqr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wirta101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'cedec001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davis102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dawsa001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'faheb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'freer101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vailm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilsg104', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'vailm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whitt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsg104', 'pitcher_hand'] = 'R'
### 1980
## AL
    events_df.loc[events_df['batterID'] == 'bochb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'castm101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'coopc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'edwad101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'edwam101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gonzd101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'burnb001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'buskt101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cleam001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cruzv101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'darwd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stons101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilbt101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'wilsw001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'yastc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wihts101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'burrr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'combg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'duesh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'figuj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fiscm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fulgj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'halie101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burrr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'combg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'duesh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fulgj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'halie101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'spikc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tekuk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'walkb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wallt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wiser101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'spikc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'tekuk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'walkb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wiser101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wohlj001', 'pitcher_hand'] = 'R'        
### 1981
## AL
    events_df.loc[events_df['batterID'] == 'bonnb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clarb002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'corct001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'davim002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dentb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'diazb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dotsr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ellij101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garcd101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'beatj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dotsr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'turnj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willd101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wrigj101', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'candj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'davic001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'davij001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'evanb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'flynd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'frant001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'heepd001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'candj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'forsb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tenag101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tianl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wiegt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'tenag101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'tianl101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wallt002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodg001', 'pitcher_hand'] = 'R'    
### 1982
## AL
    events_df.loc[events_df['batterID'] == 'bullb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'castc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'edwam102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gibsk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'castb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coopd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'eichm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'walkg001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wrigg001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'younm002', 'batter_hand'] = 'B'
## NL
    events_df.loc[events_df['batterID'] == 'cowlj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dravd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'edelj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fostg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garck001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grift101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hendg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cowlj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dravd001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'edelj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'grift101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vukog001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'walkd001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'walld001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'waltr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zuvep001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'vukog001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'walkd001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'walld001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'waltr101', 'pitcher_hand'] = 'R'
### 1983
## AL
    events_df.loc[events_df['batterID'] == 'castm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'conco001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eastj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'engld001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'espij001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'brent001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'browm003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'eastj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilst001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigr001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'younc001', 'pitcher_hand'] = 'L'    
## NL
    events_df.loc[events_df['batterID'] == 'cey-r001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'citar001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'comes001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'darlr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davig002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fimpj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'frobd001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'garnp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'citar001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'darlr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dixot101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'virgo001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vonod001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wigga001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'wilsm001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'wynnm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'virgo001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'vonod001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wigga001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willd101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wilsm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zuvep001', 'pitcher_hand'] = 'R'    
### 1984
## AL
    events_df.loc[events_df['batterID'] == 'caudb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chama001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chris001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'davia001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'davib001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'folem001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'camae001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'caudb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clibs001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'walkc001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'white002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'yoste001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willa001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younm001', 'pitcher_hand'] = 'L'    
## NL
    events_df.loc[events_df['batterID'] == 'cotth001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davie001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davig001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davim001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dedmj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ferns001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'flant001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gonzd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'davim001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dedmj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ferns001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'winnh001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zaskj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'winnh001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wockj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zaskj001', 'pitcher_hand'] = 'R'    
### 1985
## AL
    events_df.loc[events_df['batterID'] == 'butlb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coled001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dixok001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'dunbt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'bystm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clemr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'conrt001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'corbd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dixok001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ferrt001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'fingr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wehrd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yettr001', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'castr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chapk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davit001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dried001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'castr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wojne001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'worrt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'youmf001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wojne001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'worrt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wynnm001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'youmf001', 'pitcher_hand'] = 'R'    
### 1986
## AL
    events_df.loc[events_df['batterID'] == 'bushr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'caldi001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'birtt001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'clutb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coned001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'davis001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'harrg001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitl001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wittm001', 'pitcher_hand'] = 'R'  
    events_df.loc[events_df['batterID'] == 'washc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'winer001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'winnj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'walkc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'washc001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'willr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsg001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'winnj001', 'pitcher_hand'] = 'R' 
    events_df.loc[events_df['batterID'] == 'wittb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'younc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'violf001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wilkb001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wittb001', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'clarw001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'clemp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'darwd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dauld001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'daylk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dernb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'diazc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dyksl001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'folet001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'greeg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grifk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'clemp001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'daylk001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'diazc001', 'pitcher_hand'] = 'L'
### 1987
## AL
    events_df.loc[events_df['batterID'] == 'brunt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'burke001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'castj002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'downb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'flets001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clars001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'davij002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'davij003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'delel001', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'bianb001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'caryc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dowek001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fiscj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'caryc001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'fiscj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'washu001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'willf001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'young001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'younm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ziems001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'washu001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willf001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'young001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ziems001', 'pitcher_hand'] = 'R'    
### 1988
## AL
    events_df.loc[events_df['batterID'] == 'buckb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bulle001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cartj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fiskc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'castt001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'clart001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fosst001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woodm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wortc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willf002', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'biggc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'boevj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carmd001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'comsk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'costj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davir001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'diazm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dipif001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'elstk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'boevj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carmd001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'comsk001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'costj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'davir001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dipif001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woodt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wronr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younm002', 'pitcher_hand'] = 'R'    
### 1989
## AL
    events_df.loc[events_df['batterID'] == 'cabrf001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'canag001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'datzj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'finls001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'blacb001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'browk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'corbs001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'corsj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wardg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wille001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wintm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'westd001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zavac001', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'bloct001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'breas001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'canoj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'duncm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'freys001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'giraj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bloct001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'canoj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'freys001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wallt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'white003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilst002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zeilt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wallt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'white003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilst002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zeilt001', 'pitcher_hand'] = 'R'     
### 1990
## AL
    events_df.loc[events_df['batterID'] == 'boslt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'browj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'browm004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'canso001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cochd001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'galld001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'capem001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'casil001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'codic001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cumms001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gibsp001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'stonj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'younr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wickk001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'younc002', 'pitcher_hand'] = 'L'    
## NL
    events_df.loc[events_df['batterID'] == 'bondb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'branj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cedea001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'charn001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'clarm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cookd001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coraj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'dascd001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'diazm002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fletd001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'branj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'charn001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'clarm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cookd001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dascd001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'vatcj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wettj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yorkm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'vatcj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wettj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'white002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whitw001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yorkm001', 'pitcher_hand'] = 'R'
### 1991
## AL
    events_df.loc[events_df['batterID'] == 'buecs001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coles001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coops001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cronc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'boucd001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'burnt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'daltm001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'farrs001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ferna001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vizqo001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'zoske001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zupcb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wapns001', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'boner001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'camps001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'castt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'danik001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'deshd001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'drabd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'frasw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'boner001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'drabd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'frasw001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'white001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilkc001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'willm002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilss001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wohlm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodt002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'youna001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'white001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilkc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willm002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wilss001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wohlm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodt002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'youna001', 'pitcher_hand'] = 'R'
### 1992
## AL
    events_df.loc[events_df['batterID'] == 'brilg001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cansj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davia002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'deerr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eisej001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'grifa001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'hallm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'chrim001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dunnm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fettm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'flanm001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ventr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willr002', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'willm004', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodk001', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'bellj003', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'brads001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'burkj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'candc001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'candt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cartg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colev002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'coned001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cormr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'deshj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'burkj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'candt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'colev002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cormr001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'deshj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wardk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilkr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willm005', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'youne001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'younk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'younp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wardk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilkr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willm005', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younp001', 'pitcher_hand'] = 'R'     
### 1993
## AL
    events_df.loc[events_df['batterID'] == 'bross001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ceded001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'devem001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'easld001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'eldrc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'habyj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'waltj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'winfd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'waind001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whitm002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willw001', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'cadag001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'camik001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'carpc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'castp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ciana001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cooks001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cromt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dunss001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fernt001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'merco002', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'brost001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cadag001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'carpc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cooks001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsn001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'worrt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zambe001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsn001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'worrt002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zambe001', 'pitcher_hand'] = 'R'    
### 1994
## AL
    events_df.loc[events_df['batterID'] == 'brumm001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'buhnj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'disag001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'franj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gaetg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gallm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hamid001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dipoj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'erics001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'shumt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'turnc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitm002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'youne002', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wertb001', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'burbd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carrm002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dixos001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dorsb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fassj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'galaa001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'glavt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'goodd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burbd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dixos001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'fassj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'glavt001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'goodd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vizcj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'whitd002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'whitr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willm003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodb002', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'vizcj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'waltj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whitd002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whitr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodb002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'youne001', 'pitcher_hand'] = 'R'
### 1995
## AL
    events_df.loc[events_df['batterID'] == 'bradd001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'burnj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'clart002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'colev001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cordm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'boroj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bosks001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cornb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'eilad001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vitij001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitm001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'zaung001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'wolcb001', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'benzt001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'carar001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'castj004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'castv001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colbg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cornr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'courj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'donnc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'encaa001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cornr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'courj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'walkp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'walkp001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wortc001', 'pitcher_hand'] = 'R'    
### 1996
## AL
    events_df.loc[events_df['batterID'] == 'camem001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carrm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cirij001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colea001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'duram001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eenhr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'goodt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'brisj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'corba001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davic001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'listp001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'walkt002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willb002', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'waket001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wills001', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'bordp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'boroj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bottr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bours001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carrh001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'casil001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clard001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dipoj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bottr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bours001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carrh001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'taube001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wendt001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'wilsd002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilsp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'womat001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'yound001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'zubej001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'taube001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wendt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsd002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wilsp001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'womat001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yound001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zubej001', 'pitcher_hand'] = 'L'    
### 1997
## AL
    events_df.loc[events_df['batterID'] == 'bogat001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'britt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'butlr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cangj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cases001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'casta001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ceder001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'clybd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cordw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cummm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'curtc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'diaze003', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cathm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dickj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'walbm001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'widgc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'walld002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wasdj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woods001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigj002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yan-e001', 'pitcher_hand'] = 'R'   
    events_df.loc[events_df['batterID'] == 'websl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsc002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wittk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'tavaj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodk002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yoshm001', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'blaiw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bohab001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'brocd001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'carum001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'clemm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clouk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'conns001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'counc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'crabt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crour001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cruzj004', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'dehar001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'desse001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'difem001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'edmoj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fox-c001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'franm003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hawkl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'blaiw001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bohab001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'brocd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clemm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clouk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'conns001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'crabt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'crour001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dehar001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'desse001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fostk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fox-c001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hawkl001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'baerc001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'walld002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wasdj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wincs001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'winsd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wolcb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'walkt002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wincs001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'winsd001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zaung001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tavaj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wardd002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woodk002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woods001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wrigj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yount001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wardd002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'websl001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wille001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yount001', 'pitcher_hand'] = 'L'
### 1999
## AL
    events_df.loc[events_df['batterID'] == 'blauj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bushh001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'conij001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'daubb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gateb001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'browj003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'brunw001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'chrij002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cunnw001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'davej001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'durbc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'erdot001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gravd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'groob001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilsp002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wellb002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'witaj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wolfr001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'yarne001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zimmj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zimmj002', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'blumg001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'brewb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'browm005', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'burna001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'busbm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'checr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chrij002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cunnw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deltm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'derom001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gravd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'groob001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hundt001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'brewb001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'browm005', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burna001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'busbm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'checr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'deltm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'thomm002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wellb002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'witaj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wolfr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'yan-e001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yoshm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'thomm002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willr002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsp002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'youne002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zoske001', 'pitcher_hand'] = 'R'    
### 2000
## AL
    events_df.loc[events_df['batterID'] == 'burkm001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cabrj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cabro001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carpb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'damoj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'drewj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fasas001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'furcr001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'winnr001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'borkd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'buehm001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cabrj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'castf001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dingc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'downs001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'embra001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'halaj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woots001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zitob001', 'pitcher_hand'] = 'L'    
## NL
    events_df.loc[events_df['batterID'] == 'bottk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'browb003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'browk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brusj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cabrj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'castf001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'downs001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'embra001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'halaj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'bottk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'brusj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'weavj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wengd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wunsk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zerbc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zimmj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zulej001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'weavj002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wengd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'widgc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willj002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wunsk001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zerbc001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zulej001', 'pitcher_hand'] = 'R'
### 2001
## AL
    events_df.loc[events_df['batterID'] == 'branj002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'butlb002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chrim002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'clarb003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coomr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cuddm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dye-j001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'erstd001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'byrdp001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chenb001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cocop001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cubid001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'franr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whitg001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zambv001', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'breal001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'byrdp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chenb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chrir001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clemr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coolm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cotah001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cubid001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eatoa001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'franr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'breal001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'eatoa001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'waket001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitg001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zitob001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wittk001', 'pitcher_hand'] = 'R'
### 2002
## AL
    events_df.loc[events_df['batterID'] == 'berra001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'branr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'browr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'casar001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'castr002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cox-s001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'durhr001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'ellim001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hensd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cerdj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'creed001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cruzn001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'david002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'donnb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'guare001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilse001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'wilsv001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigr002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'younm003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zinta001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'towej001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wills002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zambc001', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'browa001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cerdj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chave002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cresc001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cruzn001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'custj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'donnb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fostj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'fostj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'torrs002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'walkk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wills002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zambc001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'zambv001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'torrs002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'walkk001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wilse001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsv001', 'pitcher_hand'] = 'R'    
### 2003 
## AL
    events_df.loc[events_df['batterID'] == 'bergb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carrj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clarh001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cruzd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gomec001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'boydj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cartl002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chacs001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cottn001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dejem001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'germf001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'swanp001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'thomb001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'westj001', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'budzm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bullk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chacs001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'condc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davij004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dejem001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'delov001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'estrj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'bullk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'condc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'delov001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'westj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zoccp001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'willg001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woots001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zoccp001', 'pitcher_hand'] = 'R'    
### 2004
## AL
    events_df.loc[events_df['batterID'] == 'brune001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cantj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'castl001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cinta001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'coraa001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cruzj003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ecksd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'escaf001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'granc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'biddr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'biern001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cabrd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cabrf002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chacg001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'contj002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'duchj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tifft001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'wilkb002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilsj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'youkk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'webeb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willj003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodm002', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'biddr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'biern001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'burns001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cabrd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'closj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'corrk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'darev001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dempr002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'duchj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eldrc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burns001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'corrk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'crowj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'darev001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dempr002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wheed001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willj003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willj004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilst003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wisem001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodm002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'younj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wheed001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilkb002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'willj004', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilst003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wisem001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'youkk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younj002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zinta001', 'pitcher_hand'] = 'R'    
### 2005
## AL
    events_df.loc[events_df['batterID'] == 'blalh001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'browe001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'burkc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'calzn001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cashk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clarj003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'clayr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crosb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'doumr001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'duboj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'campj002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carrg001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'davik002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willg003', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'younw001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'vasqj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'webbj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wickb001', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'bigbl001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'carrg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chava002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cruzn002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davik002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'falkb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'germf001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'falkb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vasqj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsj003', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsj002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsj003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'winnr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zimmr001', 'pitcher_hand'] = 'R'    
### 2006
## AL
    events_df.loc[events_df['batterID'] == 'byrdm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'canor001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'crosb002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'encaj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bookc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burrb001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'chulv001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'claub001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'colob001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'corct002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coreb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'craij001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'eyres001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'grilj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodj002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'yatet001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zumaj001', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'borkd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brazy001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cabrf002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cepim001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'claub001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colob001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'corct002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'craij001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'devij001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'diazv001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eyres001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'brazy001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'devij001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'venam002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yatet001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'youms001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'yound004', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'venam002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'woodj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'youms001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'younc004', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yound004', 'pitcher_hand'] = 'R'    
### 2007
## AL
    events_df.loc[events_df['batterID'] == 'buckj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'burkj003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bynuf001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cabrm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cabrm002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'credj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crisc001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'browa002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carmf001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cavaa001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cruzj005', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dejoj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dohms001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tulot001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodb003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigd002', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wasse001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wellt002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigc001', 'pitcher_hand'] = 'L'        
    events_df.loc[events_df['batterID'] == 'whitr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zaung001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'willw001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'witaj001', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'belle002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bookc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carmf001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cavaa001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chulv001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'corml001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cruzj005', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'durbc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'corml001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wised001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zagum001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zaram001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wised001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zaram001', 'pitcher_hand'] = 'R'
### 2008
## AL
    events_df.loc[events_df['batterID'] == 'bourm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'broxj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brucj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'canna001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cartc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'castk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coatb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'costc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dejed001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dillj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bellh001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'broxj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bulgj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bullb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burkj003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cappm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'crucf001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bradm001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'teixm001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'weekr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wertj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yound003', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wangc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'watec001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'weatd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigw001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'yabuy001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zinkc001', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'bellh001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bowim001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cappm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carlb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'david002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'delgj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bowim001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'carlb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'delgj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wangc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'weatd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wellk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'worrm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'weekr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wellk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wertj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'worrm001', 'pitcher_hand'] = 'R'    
### 2009
## AL
    events_df.loc[events_df['batterID'] == 'burrp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'butlb003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'canib001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carrb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cataf001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'evera001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'frank001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bowdm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burkg001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'camps002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carrc003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'casta002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'chavj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cordf002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'david004', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'burkg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'camps002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chacj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chavj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colvt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cordf002', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chacj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wolfb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodt003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zavac002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'vazqj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wellt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wolfb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'vazqj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yound003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'youne003', 'pitcher_hand'] = 'R'    
### 2010
## AL
    events_df.loc[events_df['batterID'] == 'browd003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'calla001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'casts001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'conrb001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'curtc002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'denoc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'freed001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bresc001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'byrdt001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cecib001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'coloj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'corpm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'garcf002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wiggt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wagnb001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'willr003', 'pitcher_hand'] = 'L'    
## NL
    events_df.loc[events_df['batterID'] == 'byrdt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cecib001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cedaj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cirip001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'contj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'corpm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garcf002', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cedaj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wagnb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willr003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woodt003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodt004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zawal001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'zumaj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodt004', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wrigd002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zawal001', 'pitcher_hand'] = 'R'    
### 2011
## AL
    events_df.loc[events_df['batterID'] == 'cairm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chisl001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coopd002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'deaza001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'freef001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'bedae001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'bergb002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bradd002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'bucht001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cainm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chamj002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clipt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cokep001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cuddm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'daviw001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'venaw001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'whelk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whita001', 'pitcher_hand'] = 'R'    
## NL   
    events_df.loc[events_df['batterID'] == 'bergb002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bucht001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cainm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cancr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'casta002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chama002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'clipt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cokep001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'daviw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'defrj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'dejei002', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'defrj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'velee001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'younm004', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'velee001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'venaw001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'vizqo001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younm004', 'pitcher_hand'] = 'R'    
### 2012
## AL
    events_df.loc[events_df['batterID'] == 'boesb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cainl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carpm002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'carsm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coghc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'desmi001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dunna001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gutif001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'beatp001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bracb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bundd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'caria001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carpd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carsr001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cassb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chapa001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'delad001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'delgr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'felin001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vottj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'verdr001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wilsc004', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'worlv001', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'bracb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carpd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carsr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chapa001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chatt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'choar001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'delgr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'estrm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'felin001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chatt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'choar001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'estrm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wielj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsc004', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilsj004', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'worlv001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'vottj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wielj001', 'pitcher_hand'] = 'R'    
### 2013
## AL
    events_df.loc[events_df['batterID'] == 'borbj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cabra002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cabre001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'calhk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'castj006', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'choim001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crawc002', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'bondj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'brotr001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'chenw001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cingt001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cloyt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'colel001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'corbp001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wietm001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'venty001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'webet001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wolfr002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wootr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zeidj001', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'bondj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brotr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'burgh001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chenw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cingt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cloyt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colel001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'corbp001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dewib001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'burgh001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burna002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'webet001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wolfr002', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wongk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younm003', 'pitcher_hand'] = 'R'    
### 2014    
## AL
    events_df.loc[events_df['batterID'] == 'blacc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'browd004', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'carpm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'casac001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chave001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'collt002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cruzt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dankj002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'davic003', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'boyeb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'britd001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'capuc001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cisnj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'darnl001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'drabk001', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'boyeb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'browg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deckj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'desca001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dobbg001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'falui001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'desca001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vogts001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'voger001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wagnn001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wortd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigj001', 'pitcher_hand'] = 'R'    
    events_df.loc[events_df['batterID'] == 'voger001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wootr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zeidj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yelic001', 'pitcher_hand'] = 'R'    
### 2015
## AL
    events_df.loc[events_df['batterID'] == 'avilm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'choos001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'corrc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cowgc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bella001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'buchc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carrs001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coleg001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'conla001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'duenb001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'eliar001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'vendp001', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'bogub001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bolsm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bourj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'broam001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'buchc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cahit001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cishs001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coleg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'conla001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cravt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deckc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'duenb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'bolsm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'broam001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cahit001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cishs001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cravt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'valed001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vargk001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'waldk002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wallb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'webbr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'vargk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'waldk002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wallb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'webbd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'webbr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsb002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ynoar001', 'pitcher_hand'] = 'R'    
### 2016
## AL
    events_df.loc[events_df['batterID'] == 'bourp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'branm003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bumgm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'canhm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carre001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'centj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chirr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clevs001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coatj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cozaz001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garcg002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'uribj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wendj002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willm008', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsb002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wongk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wortd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yelic001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'youne003', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'zimmr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zobrb001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'britz001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'bumgm001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'butle001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cabrm003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'campl001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cobba001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'felds001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fulmm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'velav001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wimma001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wooda002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wrigd003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ynoam001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younc003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ziegb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zimmj003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zycht001', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'buscm002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'butle001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cabrm003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'campl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'capuc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cervh001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'felds001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fulmm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gioll001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'velav001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'weekj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'wilka002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wooda002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigd003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ynoag001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ynoam001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ynoar001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'younc003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'younm005', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ziegb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zimmj003', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'buscm002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cervh001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'gioll001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willt002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ynoag001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younm005', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'uribj002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilka002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willm008', 'pitcher_hand'] = 'R'        
### 2017
## AL
    events_df.loc[events_df['batterID'] == 'blana001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'braur002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'calhw001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'castn001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cervf001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cespy001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cronc002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davij006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willm007', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willn001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woltt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'younc004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zimmb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'biagj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'brica001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cedex001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'collj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'crick001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'davij006', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dools001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'schet001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'valdc003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whitt002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodb004', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodb005', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodh001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yatek001', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'biagj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brica001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'candj002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cappc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carls002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'collj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crick001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'culbc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'darnc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilht001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilka001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wimma001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wislm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodb004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodb005', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'yatek001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zycht001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'blana001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cappc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carls002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilht001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilka001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wislm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wendj002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willn001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'woltt001', 'pitcher_hand'] = 'R'    
### 2018
## AL
    events_df.loc[events_df['batterID'] == 'albio001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'blasj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brits001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'butld001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cabrm002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'crawb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cuevn001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dejop001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dozih001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ellia001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fedet001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gonzm002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'trevj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'urshg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'villj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'vilom001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zunim001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'beekj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'biebs001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coloa001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'crock001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cruzr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'volsc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wahlb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'warra001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wendj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigs001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yacaj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yarbr001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zamod001', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'beekj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'biebs001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bourj002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bundd001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cedex001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coloa001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crock001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dools001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'marrd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'volsc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wahlb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'warra001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wendj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigs001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yarbr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zamod001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wrigk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zagum001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wadet002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yastm001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zimmb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zimmr001', 'pitcher_hand'] = 'R'
### 2019
## AL
    events_df.loc[events_df['batterID'] == 'bemba001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bryak001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'buxtb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'confm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wadel001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wadet002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wisdp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wongk002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'yastm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zavas001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'biddj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'brooa001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'casha001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'castl003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ceasd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'diekj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'fernj006', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'youna002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zeuct001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zimmk001', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'beedt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brinl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carrc003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'castl003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cordf003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cozed001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'crics001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'darnt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'diekj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fernj006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'quinr003', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'yamaj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'youna002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zeuct001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zimmk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'beedt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'crics001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'culbc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yamaj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ynoah001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zobrb001', 'pitcher_hand'] = 'R'
### 2010
## AL
    events_df.loc[events_df['batterID'] == 'browd003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'calla001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'casts001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'conrb001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'curtc002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'denoc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'freed001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bresc001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'byrdt001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cecib001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'coloj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'corpm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'garcf002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wiggt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wagnb001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'willr003', 'pitcher_hand'] = 'L'    
## NL
    events_df.loc[events_df['batterID'] == 'byrdt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cecib001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cedaj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cirip001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'contj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'corpm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garcf002', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cedaj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wagnb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willr003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woodt003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodt004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zawal001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'zumaj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodt004', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wrigd002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zawal001', 'pitcher_hand'] = 'R'    
### 2011
## AL
    events_df.loc[events_df['batterID'] == 'cairm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chisl001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coopd002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'deaza001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'freef001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'bedae001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'bergb002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bradd002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'bucht001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cainm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chamj002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clipt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cokep001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cuddm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'daviw001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'venaw001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'whelk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whita001', 'pitcher_hand'] = 'R'    
## NL   
    events_df.loc[events_df['batterID'] == 'bergb002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bucht001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cainm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cancr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'casta002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chama002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'clipt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cokep001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'daviw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'defrj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'dejei002', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'defrj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'velee001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'younm004', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'velee001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'venaw001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'vizqo001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younm004', 'pitcher_hand'] = 'R'    
### 2012
## AL
    events_df.loc[events_df['batterID'] == 'boesb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cainl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carpm002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'carsm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coghc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'desmi001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dunna001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gutif001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'beatp001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bracb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bundd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'caria001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carpd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carsr001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cassb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chapa001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'delad001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'delgr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'felin001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vottj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'verdr001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wilsc004', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'worlv001', 'pitcher_hand'] = 'R'    
## NL
    events_df.loc[events_df['batterID'] == 'bracb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carpd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carsr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chapa001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chatt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'choar001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'delgr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'estrm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'felin001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chatt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'choar001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'estrm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wielj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsc004', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilsj004', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'worlv001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'vottj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wielj001', 'pitcher_hand'] = 'R'    
### 2013
## AL
    events_df.loc[events_df['batterID'] == 'borbj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cabra002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cabre001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'calhk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'castj006', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'choim001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crawc002', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'bondj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'brotr001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'chenw001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cingt001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cloyt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'colel001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'corbp001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wietm001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'venty001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'webet001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wolfr002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wootr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zeidj001', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'bondj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brotr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'burgh001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chenw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cingt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cloyt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'colel001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'corbp001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dewib001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'burgh001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burna002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'webet001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wolfr002', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wongk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younm003', 'pitcher_hand'] = 'R'    
### 2014    
## AL
    events_df.loc[events_df['batterID'] == 'blacc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'browd004', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'carpm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'casac001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chave001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'collt002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cruzt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dankj002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'davic003', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'boyeb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'britd001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'capuc001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cisnj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'darnl001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'drabk001', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'boyeb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'browg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deckj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'desca001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dobbg001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'falui001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'desca001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vogts001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'voger001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wagnn001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wortd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigj001', 'pitcher_hand'] = 'R'    
    events_df.loc[events_df['batterID'] == 'voger001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wootr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zeidj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yelic001', 'pitcher_hand'] = 'R'    
### 2015
## AL
    events_df.loc[events_df['batterID'] == 'avilm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'choos001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'corrc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cowgc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bella001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'buchc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carrs001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coleg001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'conla001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'duenb001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'eliar001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'vendp001', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'bogub001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bolsm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bourj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'broam001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'buchc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cahit001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cishs001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coleg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'conla001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cravt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deckc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'duenb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'bolsm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'broam001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cahit001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cishs001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cravt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'valed001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vargk001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'waldk002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wallb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'webbr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'vargk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'waldk002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wallb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'webbd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'webbr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsb002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ynoar001', 'pitcher_hand'] = 'R'    
### 2016
## AL
    events_df.loc[events_df['batterID'] == 'bourp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'branm003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bumgm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'canhm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carre001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'centj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chirr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clevs001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coatj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cozaz001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garcg002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'uribj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wendj002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willm008', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsb002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wongk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wortd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yelic001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'youne003', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'zimmr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zobrb001', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'britz001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'bumgm001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'butle001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cabrm003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'campl001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cobba001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'felds001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fulmm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'velav001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wimma001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wooda002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wrigd003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ynoam001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younc003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ziegb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zimmj003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zycht001', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'buscm002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'butle001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cabrm003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'campl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'capuc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cervh001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'felds001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fulmm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gioll001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'velav001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'weekj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'wilka002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wooda002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigd003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ynoag001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ynoam001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ynoar001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'younc003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'younm005', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ziegb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zimmj003', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'buscm002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cervh001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'gioll001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willt002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ynoag001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younm005', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'uribj002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilka002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willm008', 'pitcher_hand'] = 'R'        
### 2017
## AL
    events_df.loc[events_df['batterID'] == 'blana001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'braur002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'calhw001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'castn001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cervf001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cespy001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cronc002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davij006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willm007', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willn001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woltt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'younc004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zimmb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'biagj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'brica001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cedex001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'collj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'crick001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'davij006', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dools001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'schet001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'valdc003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whitt002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodb004', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodb005', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woodh001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yatek001', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'biagj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brica001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'candj002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cappc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carls002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'collj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crick001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'culbc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'darnc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilht001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilka001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wimma001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wislm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodb004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woodb005', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'yatek001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zycht001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'blana001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cappc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carls002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilht001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilka001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wislm001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wendj002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willn001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'woltt001', 'pitcher_hand'] = 'R'    
### 2018
## AL
    events_df.loc[events_df['batterID'] == 'albio001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'blasj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brits001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'butld001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cabrm002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'crawb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cuevn001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dejop001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dozih001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ellia001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fedet001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gonzm002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'trevj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'urshg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'villj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'vilom001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zunim001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'beekj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'biebs001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coloa001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'crock001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'cruzr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'volsc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wahlb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'warra001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wendj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wrigs001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yacaj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yarbr001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zamod001', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'beekj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'biebs001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bourj002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bundd001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cedex001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coloa001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crock001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dools001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'marrd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'volsc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wahlb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'warra001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wendj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wrigs001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yarbr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zamod001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wrigk001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zagum001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wadet002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yastm001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zimmb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zimmr001', 'pitcher_hand'] = 'R'
### 2019
## AL
    events_df.loc[events_df['batterID'] == 'bemba001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bryak001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'buxtb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'confm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wadel001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wadet002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wisdp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wongk002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'yastm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zavas001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'biddj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'brooa001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'casha001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'castl003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ceasd001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'diekj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'fernj006', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'youna002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zeuct001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zimmk001', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'beedt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brinl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carrc003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'castl003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cordf003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cozed001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'crics001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'darnt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'diekj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fernj006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'quinr003', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'yamaj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'youna002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zeuct001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zimmk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'beedt001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'crics001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'culbc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yamaj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ynoah001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zobrb001', 'pitcher_hand'] = 'R'
### 2020
## AL
    events_df.loc[events_df['batterID'] == 'blant002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'brega001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carav001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'collz001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'davik003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ervip001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'estrt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wardt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'white006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'winkj002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woodj003', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bemba001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'boxbb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'brewc002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'castd002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chafa001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'civaa001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'claua001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'colea002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'duffd001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'efliz001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fergc001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'stiej001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'verlj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'votha001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilsj004', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'woodj003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'workb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yajum001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zimmb002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zubet001', 'pitcher_hand'] = 'R'
## NL
    events_df.loc[events_df['batterID'] == 'carld002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cordr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'craiw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deana001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'youna003', 'batter_hand'] = 'R'
### 2021
## AL
    events_df.loc[events_df['batterID'] == 'bellj005', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'bichb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bolts001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'boted002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brujv001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'camaj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'campl002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ciufn001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cleme002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crawj002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hamib001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'willl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsj005', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'benda001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'berrj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'brenb003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'carld003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cessl001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coled003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coons001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'urenj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'waddb001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'whitm004', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willt003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wojca001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yangh001', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'benda001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'berrj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bosta001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'boxbb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bruij001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'carld003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'claua001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'coons001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'urenj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'welkc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitm004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willt003', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'workb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yajum001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yangh001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ynoah001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zimmb002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zubet001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bosta001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bruij001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dickb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'willl001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'winkj002', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wongk002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'youna003', 'pitcher_hand'] = 'R'
### 2022
## AL
    events_df.loc[events_df['batterID'] == 'wongc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yepej001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'degrj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'weisg001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wentj001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'widet001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilec001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'wilss002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'woods002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'yound005', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zabaa001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'zastr001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'zerpa001', 'pitcher_hand'] = 'L'
## NL
    events_df.loc[events_df['batterID'] == 'younj003', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'weemj001', 'pitcher_hand'] = 'R'

    return events_df

print('ok')

ok


In [7]:
s = time.time()
events_df = events_df[events_df['event_in_game'].apply(lambda x: True == False)]
events_df = events_df_bk

print('getting roster file...')
if ((thisyear == 1914) and (league == 'F')):
    rosters_df = pandas.read_csv(rosters_dir+'rosters-orig.csv', index_col='rosterID')
elif ((thisyear > 1914) and (league == 'A')):
    #rosters_df = pandas.read_csv(rosters_dir+'rosters_through_{0:.0f}.csv'.format(thisyear-1), index_col='rosterID')
    rosters_df = pandas.read_csv(rosters_dir+'rosters.csv', index_col='rosterID')
else:
    pass
rosters_df = pandas.read_csv(rosters_dir+'rosters.csv', index_col='rosterID')
print('getting biodata...')
biodata_df = pandas.read_csv(processed_data_dir+'biodata.csv', encoding='utf-8', index_col='bioID')

#print('keeping only this year...')
#rosters_df = rosters_df[(rosters_df['theyear'] == thisyear)]

print('\tadding players not in roster file...')
rosters_df = add_special_case_roster_entries(rosters_df)

# print('backing up rosters...')
# rosters_df_bk = rosters_df


# print('adjusting events file...')
if ('theyear' not in events_df.columns):
    print('\tadding new columns for year and teams...')
    events_df = events_df.assign(theyear = pandas.to_numeric(events_df['gameID'].apply(lambda x: x[3:7]), errors='coerce', downcast='integer'))
if ('visteam' not in events_df.columns):
    events_df = events_df.assign(visteam = events_df.reset_index().merge(game_info_df.reset_index()[['gameID', 'visteam']], how='left', on='gameID').set_index('eventID')['visteam'])
if ('hometeam' not in events_df.columns):
    events_df = events_df.assign(hometeam = events_df.reset_index().merge(game_info_df.reset_index()[['gameID', 'hometeam']], how='left', on='gameID').set_index('eventID')['hometeam'])

print('\tassigning hands...')
events_df.loc[events_df['visitor_or_home'] == 0, 'batter_hand'] = events_df[events_df['visitor_or_home'] == 0].reset_index().merge(rosters_df.reset_index(), how='left', left_on=['batterID', 'theyear', 'visteam'], right_on=['playerID', 'theyear', 'theteam']).set_index('eventID')['bats']
events_df.loc[events_df['visitor_or_home'] == 1, 'batter_hand'] = events_df[events_df['visitor_or_home'] == 1].reset_index().merge(rosters_df.reset_index(), how='left', left_on=['batterID', 'theyear', 'hometeam'], right_on=['playerID', 'theyear', 'theteam']).set_index('eventID')['bats']

print('\tassigning pitcher hands...')
events_df.loc[events_df['visitor_or_home'] == 0, 'pitcher_hand'] = events_df[events_df['visitor_or_home'] == 0].reset_index().merge(rosters_df.reset_index(), how='left', left_on=['pitcherID', 'theyear', 'hometeam'], right_on=['playerID', 'theyear', 'theteam']).set_index('eventID')['throws']
events_df.loc[events_df['visitor_or_home'] == 1, 'pitcher_hand'] = events_df[events_df['visitor_or_home'] == 1].reset_index().merge(rosters_df.reset_index(), how='left', left_on=['pitcherID', 'theyear', 'visteam'], right_on=['playerID', 'theyear', 'theteam']).set_index('eventID')['throws']

print('\tfixing special case hands...')
events_df = fix_special_case_hands(events_df)

print('\taccounting for switch hitters...')
events_df.loc[(events_df['batter_hand'] == 'B') & (events_df['pitcher_hand'] == 'R'), 'batter_hand'] = 'L'
events_df.loc[(events_df['batter_hand'] == 'B') & (events_df['pitcher_hand'] == 'L'), 'batter_hand'] = 'R'

print('\tapplying adjustments to batter and pitcher hands as needed...')
#print('\n')
pitcher_hand_switcher_df = pandas.DataFrame(data=events_df[(events_df['padj'] == True)]['pitcher_hand'], columns=['pitcher_hand'])
pitcher_hand_switcher_df = pitcher_hand_switcher_df.assign(pitcher_hand_switched = np.nan)
pitcher_hand_switcher_df.loc[pitcher_hand_switcher_df['pitcher_hand'] == 'L', 'pitcher_hand_switched'] = 'R'
pitcher_hand_switcher_df.loc[pitcher_hand_switcher_df['pitcher_hand'] == 'R', 'pitcher_hand_switched'] = 'L'
events_df.loc[(events_df['padj'] == True), 'pitcher_hand'] = pitcher_hand_switcher_df['pitcher_hand_switched']

batter_hand_switcher_df = pandas.DataFrame(data=events_df[(events_df['badj'] == True)]['batter_hand'], columns=['batter_hand'])
batter_hand_switcher_df = batter_hand_switcher_df.assign(batter_hand_switched = np.nan)
batter_hand_switcher_df.loc[batter_hand_switcher_df['batter_hand'] == 'L', 'batter_hand_switched'] = 'R'
batter_hand_switcher_df.loc[batter_hand_switcher_df['batter_hand'] == 'R', 'batter_hand_switched'] = 'L'
events_df.loc[(events_df['badj'] == True), 'batter_hand'] = batter_hand_switcher_df['batter_hand_switched']

missing_hand_batters_list = events_df[events_df['batter_hand'].isnull()]['batterID'].drop_duplicates().sort_values().tolist()
missing_hand_batters_list = sorted(missing_hand_batters_list)

print('\n')
print('No batter hand found for {0:,.0f} batters...'.format(len(missing_hand_batters_list)))

missing_hand_pitchers_list = events_df[events_df['pitcher_hand'].isnull()]['pitcherID'].drop_duplicates().sort_values().tolist()
missing_hand_pitchers_list = sorted(missing_hand_pitchers_list)

print('No pitcher hand found for {0:,.0f} pitchers...'.format(len(missing_hand_pitchers_list)))
print('\n')
# for ix, thisrow in rosters_df[
#     (rosters_df['playerID'].isin(missing_hand_batters_list)) 
#     ][['playerID', 'bats']].drop_duplicates().sort_values(by='playerID').iterrows():
#     print("    events_df.loc[events_df['batterID'] == '{0:}', 'batter_hand'] = '{1:}'".format(thisrow['playerID'], thisrow['batting_hand']))

#rosters_df[(rosters_df['playerID'].isin(missing_hand_batters_list))]

print('Searching for batters and pitchers missing from roster files and/or with missing hands...')
add_to_rosters_list = []

b_events_fixer_text = ''
for this_batter in missing_hand_batters_list:
    if (this_batter not in rosters_df['playerID'].drop_duplicates().tolist()):
        #print('{0:} not in roster file'.format(this_batter))
        add_to_rosters_list.append(this_batter)
    batslist = rosters_df[rosters_df['playerID'] == this_batter]['bats'].drop_duplicates().tolist()
    if (len(batslist) == 1):
        bathand = batslist[0]
    elif (len(batslist) == 0):
        if (this_batter in biodata_df.index.tolist()):
            if (biodata_df.loc[this_batter]['bats'] in ['L', 'R', 'B', '?']):
                bathand = biodata_df.loc[this_batter]['bats']
            else:
                bathand = '?'
        else:
            bathand = '?'
    else:
        print('{0:} batted with {1:.0f} different hands: {2:}'.format(this_batter, len(batslist), batslist))
    if (bathand == '?'):
        print("\nNEED TO RESEARCH HAND FOR {0:} {1:} ({2:}) ({3:})".format(biodata_df.loc[this_batter]['nameFirst'], biodata_df.loc[this_batter]['nameLast'], this_batter, bathand))
    b_events_fixer_text += "events_df.loc[events_df['batterID'] == '{0:}', 'batter_hand'] = '{1:}'\n".format(this_batter, bathand)

p_events_fixer_text = ''
for this_pitcher in missing_hand_pitchers_list:
    if (this_pitcher not in rosters_df['playerID'].drop_duplicates().tolist()):
        add_to_rosters_list.append(this_pitcher)
    throwslist = rosters_df[rosters_df['playerID'] == this_pitcher]['throws'].drop_duplicates().tolist()
    if (len(throwslist) == 1):
        throwhand = throwslist[0]
    elif (len(throwslist) == 0):
        if (this_pitcher in biodata_df.index.tolist()):
            if (biodata_df.loc[this_pitcher]['throws'] in ['L', 'R', '?']):
                throwhand = biodata_df.loc[this_pitcher]['throws']
            else:
                throwhand = '?'
        else:
            throwhand = '?'
    else:
        print('{0:} pitched with {1:.0f} different hands: {2:}'.format(this_pitcher, len(throwslist), throwslist))
#     if (bathand == '?'):
#         print("\nNEED TO RESEARCH HAND FOR {0:} {1:} ({2:}) ({3:})".format(biodata_df.loc[this_pitcher]['nameFirst'], biodata_df.loc[this_pitcher]['nameLast'], this_pitcher, throwhand))
    p_events_fixer_text += "events_df.loc[events_df['pitcherID'] == '{0:}', 'pitcher_hand'] = '{1:}'\n".format(this_pitcher, throwhand)    

print('\n')   

positions_dict = {'field_C_playerID': 'C', 'field_1B_playerID': '1B', 'field_2B_playerID': '2B', 'field_3B_playerID': '3B', 'field_SS_playerID': 'SS', 'field_LF_playerID': 'LF', 'field_CF_playerID': 'CF', 'field_RF_playerID': 'RF', 'pitcherID': 'P', 'bat_DH_playerID': 'DH'}
# position_finder_df = pandas.DataFrame(data=None, columns=positions_dict.keys(), index=add_to_rosters_list)
# position_finder_df.loc[:, :] = 0
preferred_position = ''
roster_fixer_text = ''

for thisone in add_to_rosters_list:
    lastname = biodata_df.loc[thisone]['nameLast']
    firstname = biodata_df.loc[thisone]['nameFirst']
    if (biodata_df.loc[thisone]['bats'] in ['L', 'R', 'B', '?']):
        bathand = biodata_df.loc[thisone]['bats']
    else:
        bathand = '?'
    if (biodata_df.loc[thisone]['throws'] in ['L', 'R', '?']):
        throwhand = biodata_df.loc[thisone]['throws']
    else:
        throwhand = '?'
    teamlist = []
    teamlist += events_df[(events_df['batterID'] == thisone) & (events_df['visitor_or_home'] == 0)]['visteam'].drop_duplicates().tolist()
    teamlist += events_df[(events_df['batterID'] == thisone) & (events_df['visitor_or_home'] == 1)]['hometeam'].drop_duplicates().tolist()
    teamlist += events_df[(events_df['pitcherID'] == thisone) & (events_df['visitor_or_home'] == 1)]['visteam'].drop_duplicates().tolist()
    teamlist += events_df[(events_df['pitcherID'] == thisone) & (events_df['visitor_or_home'] == 0)]['hometeam'].drop_duplicates().tolist()
    teamlist = list(set(teamlist))
    
    for thisteam in teamlist:
        max_appearances = -1
        for k,v in positions_dict.items():
            if (len(
                    events_df[
                        (events_df[k] == thisone) 
                        & (events_df['visitor_or_home'] == 0) 
                        & (events_df['hometeam'] == thisteam)
                    ]
                ) + len(
                    events_df[
                        (events_df[k] == thisone) 
                        & (events_df['visitor_or_home'] == 0) 
                        & (events_df['hometeam'] == thisteam)
                    ]
                ) > max_appearances
            ):
                primary_position = v
                max_appearances = len(
                        events_df[
                            (events_df[k] == thisone) 
                            & (events_df['visitor_or_home'] == 0) 
                            & (events_df['hometeam'] == thisteam)
                        ]
                    ) + len(
                        events_df[
                            (events_df[k] == thisone) 
                            & (events_df['visitor_or_home'] == 0) 
                            & (events_df['hometeam'] == thisteam)
                        ]
                    )
        roster_fixer_text += "rosters_df = add_roster_entry_if_needed(rosters_df, '{0:}', '{1:}', '{2:}', '{3:}', '{4:}', '{5:}', '{6:}', {7:.0f})\n".format(thisone, lastname, firstname, bathand, throwhand, thisteam, primary_position, thisyear)

if (len(roster_fixer_text) > 0):
    print('\n')   
    print(roster_fixer_text)
if (len(b_events_fixer_text) > 0):
    print('\n')   
    print(b_events_fixer_text)
if (len(p_events_fixer_text) > 0):
    print('\n')   
    print(p_events_fixer_text)

e = time.time()
g += e-s
print('\n')
print('Assigned hands in {0:,.1f} seconds!'.format(e-s))


getting roster file...
getting biodata...
	adding players not in roster file...
	adding new columns for year and teams...
	assigning hands...
	assigning pitcher hands...
	fixing special case hands...
	accounting for switch hitters...
	applying adjustments to batter and pitcher hands as needed...


No batter hand found for 0 batters...
No pitcher hand found for 0 pitchers...


Searching for batters and pitchers missing from roster files and/or with missing hands...




Assigned hands in 18.1 seconds!


In [8]:
s = time.time()
if (league.strip() == 'N'): 
    print('saving updated roster file...')
    rosters_df.to_csv('rosters_through_{0:.0f}.csv'.format(thisyear), encoding='utf-8')

print('saving data...')
if (just_this_team_flag):
    game_info_df.to_csv('game_info_{0:}{1:}_{2:}.csv'.format(thisyear, league.lower(), just_this_team.lower()), encoding='utf-8')
    er_df.to_csv('er_{0:}{1:}_{2:}.csv'.format(thisyear, league.lower(), just_this_team.lower()), encoding='utf-8')
    events_df.to_csv('events_{0:}{1:}_{2:}.csv'.format(thisyear, league.lower(), just_this_team.lower()), encoding='utf-8')
else:
    game_info_df.to_csv('game_info_{0:}{1:}.csv'.format(thisyear, league.lower()), encoding='utf-8')
    er_df.to_csv('er_{0:}{1:}.csv'.format(thisyear, league.lower()), encoding='utf-8')
    events_df.to_csv('events_{0:}{1:}.csv'.format(thisyear, league.lower()), encoding='utf-8')

e = time.time()
g = g + (e-s)
print('Done in {0:,.0f} minutes {1:,.0f} seconds!'.format(np.floor(g/60), g%60))


saving data...
Done in 16 minutes 45 seconds!


In [9]:
FileLink(r'events_{0:.0f}{1:}.csv'.format(thisyear, league.lower()))

/kaggle/working/events_1917a.csv

In [10]:
FileLink(r'er_{0:.0f}{1:}.csv'.format(thisyear, league.lower()))

/kaggle/working/er_1917a.csv

In [11]:
FileLink(r'game_info_{0:.0f}{1:}.csv'.format(thisyear, league.lower()))

/kaggle/working/game_info_1917a.csv

In [12]:
FileLink(r'rosters_through_{0:.0f}.csv'.format(thisyear))

/kaggle/working/rosters_through_1917.csv

In [ ]:

# AL
# ANA 0
# BAL 1
# BOS 2
# CHA 3
# CLE 4
# DET 5
# HOU 6
# KCA 7
# MIN 8
# NYA 9
# OAK 10
# SEA 11
# TBA 12
# TEX 13
# TOR 14

# NL
# ARI 0
# ATL 1
# CHN 2
# CIN 3
# COL 4
# LAN 5
# MIA 6
# MIL 7
# NYN 8
# PHI 9
# PIT 10
# SDN 11
# SFN 12
# SLN 13
# WAS 14


### Batting/pitching adjustments previously processed seasons (long)

In [ ]:
# BOS 0
# CHA 1
# CLE 2
# DET 3
# NYA 4
# PHA 5
# SLA 6
# WS1 7

# BRO 0
# BSN 1
# CHN 2
# CIN 3
# NY1 4
# PHI 5
# PIT 6
# SLN 7
#events_df['badj'].dropna()

In [ ]:
# events_df = pandas.DataFrame()
# for i in range(1914,1929,1):
#     print('reading data for {0:}...'.format(i))
#     events_df_i = pandas.read_csv('/kaggle/input/baseball-events-from-retrosheetorg/{0:}rs.csv'.format(i), low_memory=False, index_col='eventID')
#     events_df = pandas.concat((events_df, events_df_i), axis=0)
# # for y in sorted(['/kaggle/input/baseball-events-from-retrosheetorg/'+x for x in os.listdir('/kaggle/input/baseball-events-from-retrosheetorg/') if (('19' in x) and ('rs.csv' in x))]):
# #     df_i = pandas.read_csv(y)

# events_df = events_df.drop([
#     'bat_DH_playerID', 'pinch_runner_playerID', 'pinch_runner_replaced_playerID',
#     'badj', 'padj', 'theyear', 'baserunning', 'play_orig', 'play_orig.1'
# ], axis=1)
# for x in ['runner_1b', 'runner_2b', 'runner_3b', 'comments']:
#     events_df.loc[:, x] = events_df[x].fillna('')
# events_df.sample(1).T